In [1]:
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime
import import_ipynb
from shared_functions import DataframeLoader, Shared
from libraries.DataframesUtils import CalculoDataframes
base_dir = 'C:\\Users\\jjgg\\Documents\\licitaciones-bbraun\\Liquidations-Incentives-Jupyter'
sub_direct = 'archivos-fuente-138'
folder_bbraun_source = os.path.join(base_dir, 'source', sub_direct)

importing Jupyter notebook from C:\Users\jjgg\Documents\licitaciones-bbraun\Liquidations-Incentives-Jupyter\libraries\DataframesUtils.ipynb


In [2]:
dataframe_loader = DataframeLoader(
    base_dir=base_dir,
    sub_direct=sub_direct
)
shared = Shared()
print(dataframe_loader.cargar_dataframes)

<bound method DataframeLoader.cargar_dataframes of <shared_functions.DataframeLoader object at 0x000001538FD90450>>


In [3]:
(
        df_resultado_detallado_previo,
        df_resultado_liquidaciones_previo,
        df_empleados,
        df_empleados_inactivos,
        df_tipos_empleados,
        df_clusters_empleado,
        df_zonas_empleado,
        df_salarios_variables,
        df_parrillas,
        df_venta_recaudo_real,
        df_venta_recaudo_presupuesto,
        df_venta_por_zona,
        df_recaudo_por_zona,
        df_rentabilidad,
        df_venta_otras_companias,
        df_recaudo_otras_companias,
        df_venta_recaudo_kams,
        df_rentabilidad_kam,
        df_venta_servicios_mtto,
        df_objetivos_cualitativos,
        df_incentivos_por_empleado,
        df_area_calculo_sba,
        df_factores_liquidacion,
        df_cluster_plan_real_recaudo,
        df_zona_plan_real_venta_ch,
        df_rentabilidad_zona,
        df_resultados_variables_cualitativas,
        df_zonas_clusters_empleado,
        df_centros_costos,
        df_empleados_centros_costos,
        df_centros_costos_grupos_productos_divisiones,
        df_parrillas_tipos_calculos,
        df_area_calculo_sba_centros_costos,
        df_renal_ambulatorio,
        codigos_ve_vc,
        _dataframes_entrada,
        fecha_liquidacion,
        meses_incentivos,
        OUTPUT_FOLDER
    ) = dataframe_loader.cargar_dataframes(base_dir, folder_bbraun_source)

In [4]:
fecha_liquidacion

'2024-05-01'

In [5]:
# maestra_resultados.to_excel('maestra_resultados_.xlsx')

In [6]:
maestra_resultados = pd.read_excel(
        io=os.path.join('maestra_resultados_.xlsx'),
        dtype={
            'Contexto': str,
            'Fecha': 'datetime64[ns]',
            'PorcentajeCumplimiento':float ,
            'Variable': str,
            'Real': float,
            'Presupuesto': float
        }
    )
maestra_resultados

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto
0,0,5004523,2024-01-01,0.80,ObjetivosCualitativos,NaN,NaN
1,1,5004523,2024-02-01,0.80,ObjetivosCualitativos,NaN,NaN
2,2,5004523,2024-03-01,0.80,ObjetivosCualitativos,NaN,NaN
3,3,5004523,2024-04-01,0.80,ObjetivosCualitativos,NaN,NaN
4,4,5004523,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN
...,...,...,...,...,...,...,...
9138,5,1010166177_2,2024-01-01,2.33,VentaFacturadaPortafolioZonaClusterSub,3.777647e+07,16190154.54
9139,6,1010166177_2,2024-02-01,7.98,VentaFacturadaPortafolioZonaClusterSub,2.583843e+08,32380309.08
9140,7,1010166177_2,2024-03-01,30.12,VentaFacturadaPortafolioZonaClusterSub,1.544331e+09,51268822.71
9141,8,1010166177_2,2024-04-01,23.10,VentaFacturadaPortafolioZonaClusterSub,1.620885e+09,70157336.34


In [7]:
# maestra_resultados_centro_costo.to_excel('maestra_resultados_centro_costo.xlsx')

In [8]:
maestra_resultados_centro_costo = pd.read_excel(
        io=os.path.join('maestra_resultados_centro_costo_.xlsx'),
        dtype={
            'Fecha': 'datetime64[ns]',
            'GrupoProducto': str,
            'Division': str,
            'CodigoCentroCosto': str,
            'Real': float,
            'Presupuesto': float ,
            'Variable': str,
            'AreaCalculo': str,
            'Consecutivo': str,            
            'TipoEmpleado': str
        }
    )

maestra_resultados_centro_costo

,Unnamed: 0,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,AreaCalculo,Consecutivo,TipoEmpleado,PorcentajeCumplimiento
0,0,5002711_1,2024-01-01,610,HC,323015040,2.838878e+08,2.926409e+08,VentaCobradaZonaPaisSub,NaN,NaN,NaN,0.970089
1,1,5002711_1,2024-01-01,620,HC,323015045,1.906658e+09,2.109654e+09,VentaCobradaZonaPaisSub,NaN,NaN,NaN,0.903778
2,2,5002711_1,2024-01-01,630,HC,323015050,1.321169e+09,1.181046e+09,VentaCobradaZonaPaisSub,NaN,NaN,NaN,1.118642
3,3,5002711_1,2024-01-01,640,HC,323015055,0.000000e+00,2.040000e+06,VentaCobradaZonaPaisSub,NaN,NaN,NaN,0.000000
4,4,5002711_1,2024-01-01,680,HC,323015055,2.727908e+08,2.324993e+08,VentaCobradaZonaPaisSub,NaN,NaN,NaN,1.173298
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10890,15,5004293_3,2024-01-01,620,HC,323015090,7.666924e+08,1.011063e+09,VentaFacturadaGrupoClientesKAMSSub,NaN,NaN,NaN,0.758303
10891,16,5004293_3,2024-02-01,620,HC,323015090,2.062303e+09,2.148509e+09,VentaFacturadaGrupoClientesKAMSSub,NaN,NaN,NaN,0.959876
10892,17,5004293_3,2024-03-01,620,HC,323015090,3.146574e+09,3.412338e+09,VentaFacturadaGrupoClientesKAMSSub,NaN,NaN,NaN,0.922117
10893,18,5004293_3,2024-04-01,620,HC,323015090,4.339242e+09,4.454997e+09,VentaFacturadaGrupoClientesKAMSSub,NaN,NaN,NaN,0.974017


In [9]:
maestra_resultados_centro_costo['AreaCalculo'].fillna(0, inplace=True)
maestra_resultados_centro_costo['AreaCalculo'] = maestra_resultados_centro_costo['AreaCalculo'].astype(int)
maestra_resultados_centro_costo

,Unnamed: 0,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,AreaCalculo,Consecutivo,TipoEmpleado,PorcentajeCumplimiento
0,0,5002711_1,2024-01-01,610,HC,323015040,2.838878e+08,2.926409e+08,VentaCobradaZonaPaisSub,0,NaN,NaN,0.970089
1,1,5002711_1,2024-01-01,620,HC,323015045,1.906658e+09,2.109654e+09,VentaCobradaZonaPaisSub,0,NaN,NaN,0.903778
2,2,5002711_1,2024-01-01,630,HC,323015050,1.321169e+09,1.181046e+09,VentaCobradaZonaPaisSub,0,NaN,NaN,1.118642
3,3,5002711_1,2024-01-01,640,HC,323015055,0.000000e+00,2.040000e+06,VentaCobradaZonaPaisSub,0,NaN,NaN,0.000000
4,4,5002711_1,2024-01-01,680,HC,323015055,2.727908e+08,2.324993e+08,VentaCobradaZonaPaisSub,0,NaN,NaN,1.173298
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10890,15,5004293_3,2024-01-01,620,HC,323015090,7.666924e+08,1.011063e+09,VentaFacturadaGrupoClientesKAMSSub,0,NaN,NaN,0.758303
10891,16,5004293_3,2024-02-01,620,HC,323015090,2.062303e+09,2.148509e+09,VentaFacturadaGrupoClientesKAMSSub,0,NaN,NaN,0.959876
10892,17,5004293_3,2024-03-01,620,HC,323015090,3.146574e+09,3.412338e+09,VentaFacturadaGrupoClientesKAMSSub,0,NaN,NaN,0.922117
10893,18,5004293_3,2024-04-01,620,HC,323015090,4.339242e+09,4.454997e+09,VentaFacturadaGrupoClientesKAMSSub,0,NaN,NaN,0.974017


In [10]:
maestra_resultados_centro_costo[maestra_resultados_centro_costo['Variable'] == 'RentabilidadUnidadNegocioCM2COGSSUB']

,Unnamed: 0,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,AreaCalculo,Consecutivo,TipoEmpleado,PorcentajeCumplimiento
9738,0,1013583607_1,2024-03-01,805,AE,323007280,172273.580838,169713.102995,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,1.015087
9739,1,1013583607_1,2024-03-01,805,AE,323007290,172273.580838,169713.102995,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,1.015087
9740,2,1013583607_1,2024-04-01,805,AE,323007280,415179.066504,401140.061624,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,1.034998
9741,3,1013583607_1,2024-04-01,805,AE,323007290,415179.066504,401140.061624,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,1.034998
9742,4,1013583607_1,2024-05-01,805,AE,323007280,716717.704506,694280.875887,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,1.032317
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9835,97,94460154_2,2024-05-01,A33,BA,323007337,23399.712828,20723.216486,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,1.129154
9836,98,94460154_2,2024-05-01,A34,BA,323007337,470369.985379,677503.485037,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,0.694269
9837,99,94460154_3,2024-03-01,A35,BA,323007337,64788.652349,110600.891998,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,0.585788
9838,100,94460154_3,2024-04-01,A35,BA,323007337,153573.270718,276453.474670,RentabilidadUnidadNegocioCM2COGSSUB,0,NaN,NaN,0.555512


## Euros

In [11]:
def convertir_a_euros(main):
    # Definir las tasas de cambio
    trm_marzo_2024 = 4253.92
    trm_abril_2024 = 4226.47
    trm_promedio_2024 = 4652.35

    # Crear una copia del DataFrame para evitar el SettingWithCopyWarning
    main = main.copy()

    # Convertir los valores de 'Real' y 'Presupuesto' a euros
    main['RealEuros'] = main['Real'] / trm_abril_2024
    main['PresupuestoEuros'] = main['Presupuesto'] / trm_promedio_2024
    main['PorcentajeCumplimientoEuros'] = (main['RealEuros']/main['PresupuestoEuros']).apply(lambda x: round(x, 2))

    # Renombrar las columnas originales 'Real' y 'Presupuesto'
    main = main.rename(columns={
        'Real': 'RealCOP', 'Presupuesto': 'PresupuestoCOP', 
        'PorcentajeCumplimiento': 'PorcentajeCumplimientoCOP',
        'PorcentajeCumplimientoEuros': 'PorcentajeCumplimiento'
    })

    # Renombrar las nuevas columnas para reflejar que están en euros
    main = main.rename(columns={'RealEuros': 'Real', 'PresupuestoEuros': 'Presupuesto'})
    return main[['Contexto', 'Variable', 'Fecha', 'Real', 'Presupuesto', 'PorcentajeCumplimiento', 'RealCOP', 'PresupuestoCOP', 'PorcentajeCumplimientoCOP']]

In [12]:
def unir_dataframes_por_columnas_comunes(df1, df2, how='inner'):
    """
    Une dos DataFrames basándose en sus columnas comunes con un tipo de unión especificado.
    
    Parámetros:
    - df1: Primer DataFrame.
    - df2: Segundo DataFrame.
    - how: Tipo de unión ('left', 'right', 'outer', 'inner'). Por defecto es 'inner'.
    
    Retorna:
    - DataFrame resultante de la unión.
    """
    # Encuentra las columnas comunes entre ambos DataFrames
    columnas_comunes = df1.columns.intersection(df2.columns)
    print(columnas_comunes)
    # Une los DataFrames basándose en las columnas comunes y el tipo de unión especificado
    df_unido = pd.merge(df1, df2, on=columnas_comunes.tolist(), how=how)
    
    return df_unido

In [13]:
# variables_convertir_euros = ['VentaFacturadaBraunPais', 'VentaFacturadaZonaPaisSub', 'VentaFacturadaDivisionSub', 'VentaFacturadaUnidadNegocioSub', 'VentaFacturadaGrupoClientesKAMSSub']
# variables_convertir_euros

In [14]:
# maestra_resultados_aplicar_euros = maestra_resultados[maestra_resultados['Variable'].isin(variables_convertir_euros)]
# maestra_resultados_aplicar_euros

In [15]:
# maestra_resultados[maestra_resultados['Variable'] == 'VentaFacturadaBraunPais']

In [16]:
# maestra_resultados_convertido_euros = maestra_resultados.copy()

# maestra_resultados_convertido_euros['RealCOP'] = 0
# maestra_resultados_convertido_euros['PresupuestoCOP'] = 0
# maestra_resultados_convertido_euros['PorcentajeCumplimientoCOP'] = 0

# maestra_resultados_convertido_euros.update(convertir_a_euros(maestra_resultados_aplicar_euros))
# maestra_resultados_convertido_euros

In [17]:
# maestra_resultados_convertido_euros[maestra_resultados_convertido_euros['Variable'] == 'VentaFacturadaBraunPais']

In [18]:
# # Renombrar las nuevas columnas para reflejar que están en euros
# main = main.rename(columns={'RealEuros': 'Real', 'PresupuestoEuros': 'Presupuesto'})

In [19]:
# # Renombrar las columnas originales 'Real' y 'Presupuesto'
# main = main.rename(columns={
#     'Real': 'RealCOP', 'Presupuesto': 'PresupuestoCOP', 
#     'PorcentajeCumplimiento': 'PorcentajeCumplimientoCOP',
#     'PorcentajeCumplimientoEuros': 'PorcentajeCumplimiento'
# })

In [20]:
# maestra_resultados_convertidos = maestra_resultados.drop(columns=['Unnamed: 0']).merge(
#     maestra_resultados_convertido_euros,
#     on = ['Contexto', 'Variable', 'Fecha'],
#     how='left'
# )
# maestra_resultados_convertidos

In [21]:
# maestra_resultados_convertidos[maestra_resultados_convertidos['Variable'] == 'VentaFacturadaBraunPais']

In [22]:
# maestra_resultados = maestra_resultados_convertido_euros.copy()
# maestra_resultados

## Se recalculan los porcentajes de cumplimiento para darle manejo a negativos

In [23]:
df_parrillas[df_parrillas['Variable'] == 'VentaCobradaBraunPais']

,CodigoEmpleado,Variable,Contexto,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo
0,5004523,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN
9,1144160710,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN
17,1111128,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN
39,5004589,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN
45,5032450,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN
...,...,...,...,...,...,...,...
1305,5041369,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN
1319,5005917,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN
1324,5041312,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN
1332,1111118,VentaCobradaBraunPais,Pais,0.3,0,Venta cobrada país,NaN


In [24]:
calculo_dataframe = CalculoDataframes()

variables_convertir_euros = ['VentaFacturadaBraunPais', 'VentaFacturadaZonaPaisSub', 'VentaFacturadaDivisionSub', 'VentaFacturadaUnidadNegocioSub', 'VentaFacturadaGrupoClientesKAMSSub']
# Convertir las variables a euros en 'maestra_resultados'
maestra_resultados = calculo_dataframe.convertir_valores_a_euros(maestra_resultados, variables_convertir_euros)

# Convertir las variables a euros en 'maestra_resultados_centro_costo'
maestra_resultados_centro_costo = calculo_dataframe.convertir_valores_a_euros(maestra_resultados_centro_costo, variables_convertir_euros)

# ## Se recalculan los porcentajes de cumplimiento para darle manejo a negativos

In [25]:
def recalcular_porcentajes_cumplimiento(row):
    response = row['PorcentajeCumplimiento']
    if not pd.isnull(row['Real']) and not pd.isnull(row['Presupuesto']):
        if row['Presupuesto'] == 0:
            response = 0
        elif row['Real'] - row['Presupuesto'] < 0:
            response = 1 + ((row['Real'] - row['Presupuesto']) / abs(row['Presupuesto']))
        elif row['Real'] - row['Presupuesto'] > 0:
            response = 1 + abs((row['Real'] - row['Presupuesto']) / abs(row['Presupuesto']))
    return round(response, 2)

maestra_resultados['PorcentajeCumplimiento'] = maestra_resultados.apply(recalcular_porcentajes_cumplimiento, axis=1)
maestra_resultados

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP
0,0,5004523,2024-01-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0
1,1,5004523,2024-02-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0
2,2,5004523,2024-03-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0
3,3,5004523,2024-04-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0
4,4,5004523,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
9138,5,1010166177_2,2024-01-01,2.33,VentaFacturadaPortafolioZonaClusterSub,3.777647e+07,16190154.54,0.0,0.0,0.0
9139,6,1010166177_2,2024-02-01,7.98,VentaFacturadaPortafolioZonaClusterSub,2.583843e+08,32380309.08,0.0,0.0,0.0
9140,7,1010166177_2,2024-03-01,30.12,VentaFacturadaPortafolioZonaClusterSub,1.544331e+09,51268822.71,0.0,0.0,0.0
9141,8,1010166177_2,2024-04-01,23.10,VentaFacturadaPortafolioZonaClusterSub,1.620885e+09,70157336.34,0.0,0.0,0.0


### Se calculan los porcentajes de cumplimiento para Maestra de Centro de Costos

In [26]:
# maestra_resultados_centro_costo['PorcentajeCumplimiento'] = 0
maestra_resultados_centro_costo['PorcentajeCumplimiento'] = maestra_resultados_centro_costo.apply(recalcular_porcentajes_cumplimiento, axis=1)
# maestra_resultados_centro_costo.drop(columns={'AreaCalculo', 'Consecutivo', 'TipoEmpleado'})

## Se une la maestra de resultados con la parrilla

In [27]:
maestra_resultados[
     (maestra_resultados['Fecha']=='2024-05-01') &
    (maestra_resultados['Contexto']=='1111117')
]

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP
514,514,1111117,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0
724,24,1111117,2024-05-01,0.88,VentaFacturadaZonaCargoIndividual,3.179236e+08,3.617851e+08,0.0,0.0,0.0
2054,24,1111117,2024-05-01,0.79,VentasCobradaZonaCargoIndividual,3.044938e+08,3.835532e+08,0.0,0.0,0.0
4350,29,1111117,2024-05-01,0.86,VentaCobradaBraunCluster,3.632880e+08,4.212271e+08,0.0,0.0,0.0
5443,24,1111117,2024-05-01,0.88,VentaFacturadaPortafolioCluster,3.179236e+08,3.617851e+08,0.0,0.0,0.0
6797,39,1111117,2024-05-01,0.94,VentaFacturadaZonaPais,5.855320e+10,6.258874e+10,0.0,0.0,0.0
7117,4,1111117,2024-05-01,3.37,VentaFacturadaPortafolioRenal,1.814526e+09,5.383562e+08,0.0,0.0,0.0
7676,24,1111117,2024-05-01,0.86,VentaCobradaPortafolioCluster,3.632880e+08,4.212271e+08,0.0,0.0,0.0
9127,4,1111117,2024-05-01,1.04,VentaCobradaPortafolioRenal,8.571691e+08,8.276971e+08,0.0,0.0,0.0


In [28]:
# df_parrillas[df_parrillas['Variable'] == 'RentabilidadDivisionCM2COGsSub']

In [29]:
df_parrillas_tipos_calculos.head()

,TipoEmpleado,AreaCalculo,CodigoRubro,NombreRubro,TipoCalculo
0,AD,1,2,EBITCOGSPaisSUB,F
1,AD,1,3,ObjetivosCualitativos,F
2,AD,1,1,VentaFacturadaBraunPaisSUB,F
3,AD,2,2,EBITCOGSPaisSUB,F
4,AD,2,3,ObjetivosCualitativos,F


In [30]:
df_parrillas[df_parrillas['CodigoEmpleado'] == '5004523']

,CodigoEmpleado,Variable,Contexto,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo
0,5004523,VentaCobradaBraunPais,Pais,0.30,0,Venta cobrada país,NaN
1,5004523,ObjetivosCualitativos,5004523,0.15,0,Objetivos cualitativos,NaN
2,5004523,VentaFacturadaPortafolioClusterSub,5004523_1,0.06,1,VentaFacturadaPortafolioClusterSub,Surgery & Interventions (Neuro)
3,5004523,VentaFacturadaPortafolioClusterSub,5004523_2,0.06,2,VentaFacturadaPortafolioClusterSub,Surgery & Interventions (Ortho)
4,5004523,VentaFacturadaPortafolioClusterSub,5004523_3,0.06,3,VentaFacturadaPortafolioClusterSub,Surgery & Interventions (Surgical Assets)
5,5004523,VentaFacturadaPortafolioClusterSub,5004523_4,0.06,4,VentaFacturadaPortafolioClusterSub,Surgery & Interventions (Operating Room Soluti...
6,5004523,VentaFacturadaPortafolioClusterSub,5004523_5,0.03,5,VentaFacturadaPortafolioClusterSub,Patient Care (Desinfeccion)
7,5004523,VentaFacturadaPortafolioClusterSub,5004523_6,0.03,6,VentaFacturadaPortafolioClusterSub,Patient Care (Heridas+Ostomia)
8,5004523,VentaCobradaBraunCluster,5004523,0.25,0,Venta Cobrada presupuesto asignado total B. Br...,NaN


In [31]:
maestra_resultados[maestra_resultados['Variable'] == 'VentaCobradaBraunPais']

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP
3400,0,Pais,2024-01-01,0.95,VentaCobradaBraunPais,1.670438e+10,1.749805e+10,0.0,0.0,0.0
3401,1,Pais,2024-02-01,0.88,VentaCobradaBraunPais,3.221882e+10,3.663783e+10,0.0,0.0,0.0
3402,2,Pais,2024-03-01,0.86,VentaCobradaBraunPais,4.803727e+10,5.604000e+10,0.0,0.0,0.0
3403,3,Pais,2024-04-01,0.87,VentaCobradaBraunPais,6.413254e+10,7.395302e+10,0.0,0.0,0.0
3404,4,Pais,2024-05-01,0.91,VentaCobradaBraunPais,8.319083e+10,9.187600e+10,0.0,0.0,0.0


In [32]:
maestra_resultados_centro_costo[maestra_resultados_centro_costo['Variable'] == 'VentaFacturadaPortafolioClusterSub']

,Unnamed: 0,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,AreaCalculo,Consecutivo,TipoEmpleado,PorcentajeCumplimiento,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP
4372,0,1111111_1,2024-01-01,805,AE,323007332,7.798772e+07,1.288839e+07,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,6.05,0.0,0.0,0.0
4373,1,1111111_1,2024-01-01,805,AE,323007333,7.798772e+07,1.288839e+07,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,6.05,0.0,0.0,0.0
4374,2,1111111_1,2024-01-01,805,AE,323010170,7.798772e+07,1.288839e+07,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,6.05,0.0,0.0,0.0
4375,3,1111111_1,2024-01-01,M06,AE,323007332,2.362315e+06,1.075307e+07,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,0.22,0.0,0.0,0.0
4376,4,1111111_1,2024-01-01,M06,AE,323007333,2.362315e+06,1.075307e+07,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,0.22,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8573,4201,53069050_3,2024-05-01,610,HC,323015095,1.341899e+08,1.850790e+08,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,0.73,0.0,0.0,0.0
8574,4202,79625102_1,2024-02-01,A35,BA,323007330,2.341397e+08,2.680889e+08,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,0.87,0.0,0.0,0.0
8575,4203,79625102_1,2024-03-01,A35,BA,323007330,3.244427e+08,5.808592e+08,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,0.56,0.0,0.0,0.0
8576,4204,79625102_1,2024-04-01,A35,BA,323007330,4.592916e+08,9.383110e+08,VentaFacturadaPortafolioClusterSub,0,NaN,NaN,0.49,0.0,0.0,0.0


In [33]:
df_parrillas[
     (df_parrillas['Variable'] == 'RentabilidadUnidadNegocioCM2COGSSUB') 
#     (df_parrillas['CodigoEmpleado'] == '5036358')
]

,CodigoEmpleado,Variable,Contexto,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo
36,53069050,RentabilidadUnidadNegocioCM2COGSSUB,53069050_1,0.04,1,RentabilidadUnidadNegocioCM2COGSSUB,Grupo Local 640 Enteral Diets
37,53069050,RentabilidadUnidadNegocioCM2COGSSUB,53069050_2,0.10,2,RentabilidadUnidadNegocioCM2COGSSUB,Grupo Local 680 Parenteral Nutrition
38,53069050,RentabilidadUnidadNegocioCM2COGSSUB,53069050_3,0.06,3,RentabilidadUnidadNegocioCM2COGSSUB,Grupo Local 610 IV Drugs & LVP
254,5004297,RentabilidadUnidadNegocioCM2COGSSUB,5004297_1,0.10,1,RentabilidadUnidadNegocioCM2COGSSUB,ORS (003)
255,5004297,RentabilidadUnidadNegocioCM2COGSSUB,5004297_2,0.09,2,RentabilidadUnidadNegocioCM2COGSSUB,MIS (801)
256,5004297,RentabilidadUnidadNegocioCM2COGSSUB,5004297_3,0.01,3,RentabilidadUnidadNegocioCM2COGSSUB,iVT 006
302,91285756,RentabilidadUnidadNegocioCM2COGSSUB,91285756_1,0.06,1,RentabilidadUnidadNegocioCM2COGSSUB,8 IV Systems & Drainage
303,91285756,RentabilidadUnidadNegocioCM2COGSSUB,91285756_2,0.13,2,RentabilidadUnidadNegocioCM2COGSSUB,12 Automated Infusion Systems & Disposablesm+ ...
304,91285756,RentabilidadUnidadNegocioCM2COGSSUB,91285756_3,0.01,3,RentabilidadUnidadNegocioCM2COGSSUB,SBA02 Drugs
360,5004306,RentabilidadUnidadNegocioCM2COGSSUB,5004306_1,0.10,1,RentabilidadUnidadNegocioCM2COGSSUB,ORS (003)


In [34]:
maestra_resultados[maestra_resultados['Contexto'] == '5002711_1'].merge(
    df_parrillas[df_parrillas['Contexto'] == '5002711_1'],
    on=['Variable', 'Contexto'],
    how='left'
)

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,CodigoEmpleado,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo
0,0,5002711_1,2024-01-01,0.99,VentaCobradaZonaPaisSub,3.784505e+09,3.817880e+09,0.000000e+00,0.000000e+00,0.00,NaN,NaN,NaN,NaN,NaN
1,1,5002711_1,2024-02-01,0.99,VentaCobradaZonaPaisSub,7.780378e+09,7.846545e+09,0.000000e+00,0.000000e+00,0.00,NaN,NaN,NaN,NaN,NaN
2,2,5002711_1,2024-03-01,0.94,VentaCobradaZonaPaisSub,1.117600e+10,1.187179e+10,0.000000e+00,0.000000e+00,0.00,NaN,NaN,NaN,NaN,NaN
3,3,5002711_1,2024-04-01,0.96,VentaCobradaZonaPaisSub,1.488131e+10,1.546509e+10,0.000000e+00,0.000000e+00,0.00,NaN,NaN,NaN,NaN,NaN
4,4,5002711_1,2024-05-01,0.99,VentaCobradaZonaPaisSub,1.878196e+10,1.905839e+10,0.000000e+00,0.000000e+00,0.00,NaN,NaN,NaN,NaN,NaN
5,0,5002711_1,2024-01-01,1.37,VentaFacturadaZonaPaisSub,7.803963e+05,5.712712e+05,3.298322e+09,2.657754e+09,1.24,5002711,0.26,1.0,VentaFacturadaZonaPaisSub,Hospital Care
6,9,5002711_1,2024-02-01,1.09,VentaFacturadaZonaPaisSub,1.370481e+06,1.254635e+06,5.792297e+09,5.837001e+09,0.99,5002711,0.26,1.0,VentaFacturadaZonaPaisSub,Hospital Care
7,18,5002711_1,2024-03-01,1.11,VentaFacturadaZonaPaisSub,2.336360e+06,2.103221e+06,9.874553e+09,9.784922e+09,1.01,5002711,0.26,1.0,VentaFacturadaZonaPaisSub,Hospital Care
8,27,5002711_1,2024-04-01,1.18,VentaFacturadaZonaPaisSub,3.266975e+06,2.761944e+06,1.380777e+10,1.284953e+10,1.07,5002711,0.26,1.0,VentaFacturadaZonaPaisSub,Hospital Care
9,36,5002711_1,2024-05-01,1.17,VentaFacturadaZonaPaisSub,4.218864e+06,3.612253e+06,1.783090e+10,1.680546e+10,1.06,5002711,0.26,1.0,VentaFacturadaZonaPaisSub,Hospital Care


### Maestra de Resultados por Empleado

In [35]:
maestra_resultados_por_empleado = maestra_resultados.merge(
    df_parrillas,
    how='left',
    indicator=True
)
maestra_resultados_por_empleado

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,CodigoEmpleado,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,_merge
0,0,5004523,2024-01-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,5004523,0.15,0.0,Objetivos cualitativos,NaN,both
1,1,5004523,2024-02-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,5004523,0.15,0.0,Objetivos cualitativos,NaN,both
2,2,5004523,2024-03-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,5004523,0.15,0.0,Objetivos cualitativos,NaN,both
3,3,5004523,2024-04-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,5004523,0.15,0.0,Objetivos cualitativos,NaN,both
4,4,5004523,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,5004523,0.15,0.0,Objetivos cualitativos,NaN,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10497,5,1010166177_2,2024-01-01,2.33,VentaFacturadaPortafolioZonaClusterSub,3.777647e+07,16190154.54,0.0,0.0,0.0,1010166177,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both
10498,6,1010166177_2,2024-02-01,7.98,VentaFacturadaPortafolioZonaClusterSub,2.583843e+08,32380309.08,0.0,0.0,0.0,1010166177,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both
10499,7,1010166177_2,2024-03-01,30.12,VentaFacturadaPortafolioZonaClusterSub,1.544331e+09,51268822.71,0.0,0.0,0.0,1010166177,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both
10500,8,1010166177_2,2024-04-01,23.10,VentaFacturadaPortafolioZonaClusterSub,1.620885e+09,70157336.34,0.0,0.0,0.0,1010166177,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both


In [36]:
maestra_resultados_por_empleado[
     (maestra_resultados_por_empleado['Fecha']=='2024-05-01') &
    (maestra_resultados_por_empleado['Contexto']=='1111117')
]

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,CodigoEmpleado,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,_merge
514,514,1111117,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,1111117,0.15,0.0,Objetivos cualitativos,NaN,both
724,24,1111117,2024-05-01,0.88,VentaFacturadaZonaCargoIndividual,3.179236e+08,3.617851e+08,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,left_only
2054,24,1111117,2024-05-01,0.79,VentasCobradaZonaCargoIndividual,3.044938e+08,3.835532e+08,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,left_only
5415,29,1111117,2024-05-01,0.86,VentaCobradaBraunCluster,3.632880e+08,4.212271e+08,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,left_only
6508,24,1111117,2024-05-01,0.88,VentaFacturadaPortafolioCluster,3.179236e+08,3.617851e+08,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,left_only
7972,39,1111117,2024-05-01,0.94,VentaFacturadaZonaPais,5.855320e+10,6.258874e+10,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,left_only
8292,4,1111117,2024-05-01,3.37,VentaFacturadaPortafolioRenal,1.814526e+09,5.383562e+08,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,left_only
8851,24,1111117,2024-05-01,0.86,VentaCobradaPortafolioCluster,3.632880e+08,4.212271e+08,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,left_only
10486,4,1111117,2024-05-01,1.04,VentaCobradaPortafolioRenal,8.571691e+08,8.276971e+08,0.0,0.0,0.0,1111117,0.25,0.0,VentaCobradaPortafolioRenal,NaN,both


In [37]:
df_empleados[df_empleados['CodigoEmpleado'] == '1111117']

,TipoEmpleado,CodigoEmpleado,Identificacion,CorreoElectronico,Nombre,Apellidos,SalarioVariable,FechaIngreso,FechaRetiro,AreaCalculo
102,VC,1111117,1014226605,julieth.pedraza@bbraun.com,Julieth,Pedraza,2500000.0,2023-01-01,NaT,5


In [38]:
maestra_resultados_por_empleado[maestra_resultados_por_empleado['Variable'] == 'VentaFacturadaGrupoClientesKAMSSub']

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,CodigoEmpleado,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,_merge
8788,0,kam_2_1,2024-01-01,1.04,VentaFacturadaGrupoClientesKAMSSub,1.579665e+05,1.517270e+05,6.676407e+08,7.058871e+08,0.95,5004293,0.06,1.0,VentaFacturadaGrupoClientesKAMSSub,SBA 08 IV Systems & Drainage,both
8789,1,kam_2_1,2024-02-01,0.91,VentaFacturadaGrupoClientesKAMSSub,3.274076e+05,3.583340e+05,1.383778e+09,1.667095e+09,0.83,5004293,0.06,1.0,VentaFacturadaGrupoClientesKAMSSub,SBA 08 IV Systems & Drainage,both
8790,2,kam_2_1,2024-03-01,0.88,VentaFacturadaGrupoClientesKAMSSub,5.873638e+05,6.650162e+05,2.482475e+09,3.093888e+09,0.80,5004293,0.06,1.0,VentaFacturadaGrupoClientesKAMSSub,SBA 08 IV Systems & Drainage,both
8791,3,kam_2_1,2024-04-01,0.93,VentaFacturadaGrupoClientesKAMSSub,8.059016e+05,8.651667e+05,3.406119e+09,4.025058e+09,0.85,5004293,0.06,1.0,VentaFacturadaGrupoClientesKAMSSub,SBA 08 IV Systems & Drainage,both
8792,4,kam_2_1,2024-05-01,0.91,VentaFacturadaGrupoClientesKAMSSub,1.036601e+06,1.136338e+06,4.381161e+09,5.286644e+09,0.83,5004293,0.06,1.0,VentaFacturadaGrupoClientesKAMSSub,SBA 08 IV Systems & Drainage,both
8793,5,kam_2_2,2024-01-01,1.49,VentaFacturadaGrupoClientesKAMSSub,5.477026e+05,3.682217e+05,2.314848e+09,1.713096e+09,1.35,5004293,0.10,2.0,VentaFacturadaGrupoClientesKAMSSub,SBA 12 Automated Infusion Systems & Disposables,both
8794,6,kam_2_2,2024-02-01,1.30,VentaFacturadaGrupoClientesKAMSSub,1.130694e+06,8.696299e+05,4.778845e+09,4.045823e+09,1.18,5004293,0.10,2.0,VentaFacturadaGrupoClientesKAMSSub,SBA 12 Automated Infusion Systems & Disposables,both
8795,7,kam_2_2,2024-03-01,1.16,VentaFacturadaGrupoClientesKAMSSub,1.873040e+06,1.613908e+06,7.916346e+09,7.508463e+09,1.05,5004293,0.10,2.0,VentaFacturadaGrupoClientesKAMSSub,SBA 12 Automated Infusion Systems & Disposables,both
8796,8,kam_2_2,2024-04-01,1.22,VentaFacturadaGrupoClientesKAMSSub,2.569475e+06,2.099647e+06,1.085981e+10,9.768292e+09,1.11,5004293,0.10,2.0,VentaFacturadaGrupoClientesKAMSSub,SBA 12 Automated Infusion Systems & Disposables,both
8797,9,kam_2_2,2024-05-01,1.21,VentaFacturadaGrupoClientesKAMSSub,3.337287e+06,2.757745e+06,1.410494e+10,1.283000e+10,1.10,5004293,0.10,2.0,VentaFacturadaGrupoClientesKAMSSub,SBA 12 Automated Infusion Systems & Disposables,both


In [39]:
maestra_resultados_por_empleado = maestra_resultados_por_empleado.merge(
    df_empleados[['CodigoEmpleado', 'TipoEmpleado', 'AreaCalculo', 'Apellidos', 'Nombre', 'FechaIngreso']],
    on='CodigoEmpleado',
    how='left'
)
maestra_resultados_por_empleado

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,_merge,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso
0,0,5004523,2024-01-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
1,1,5004523,2024-02-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
2,2,5004523,2024-03-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
3,3,5004523,2024-04-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
4,4,5004523,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10497,5,1010166177_2,2024-01-01,2.33,VentaFacturadaPortafolioZonaClusterSub,3.777647e+07,16190154.54,0.0,0.0,0.0,...,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01
10498,6,1010166177_2,2024-02-01,7.98,VentaFacturadaPortafolioZonaClusterSub,2.583843e+08,32380309.08,0.0,0.0,0.0,...,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01
10499,7,1010166177_2,2024-03-01,30.12,VentaFacturadaPortafolioZonaClusterSub,1.544331e+09,51268822.71,0.0,0.0,0.0,...,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01
10500,8,1010166177_2,2024-04-01,23.10,VentaFacturadaPortafolioZonaClusterSub,1.620885e+09,70157336.34,0.0,0.0,0.0,...,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01


In [40]:
maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['Variable'] == 'ObjetivosCualitativos') &
    (maestra_resultados_por_empleado['Contexto'] == '5004293')
]

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,_merge,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso
625,625,5004293,2024-01-01,0.8,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.3,0.0,Objetivos cualitativos,NaN,both,KAM,2.0,SERNA CASTRILLON,ANGELA MARIA,2013-06-17
626,626,5004293,2024-02-01,0.8,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.3,0.0,Objetivos cualitativos,NaN,both,KAM,2.0,SERNA CASTRILLON,ANGELA MARIA,2013-06-17
627,627,5004293,2024-03-01,0.8,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.3,0.0,Objetivos cualitativos,NaN,both,KAM,2.0,SERNA CASTRILLON,ANGELA MARIA,2013-06-17
628,628,5004293,2024-04-01,0.8,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.3,0.0,Objetivos cualitativos,NaN,both,KAM,2.0,SERNA CASTRILLON,ANGELA MARIA,2013-06-17
629,629,5004293,2024-05-01,0.8,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.3,0.0,Objetivos cualitativos,NaN,both,KAM,2.0,SERNA CASTRILLON,ANGELA MARIA,2013-06-17


In [41]:
# Se añaden líneas para filtrar registros vacíos ('Fecha' y 'CodigoEmpleado')
maestra_resultados_por_empleado.dropna(subset=[
        'Fecha'
    ], inplace=True)
maestra_resultados_por_empleado['CodigoEmpleado'] = maestra_resultados_por_empleado['CodigoEmpleado'].astype(str)
maestra_resultados_por_empleado

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,_merge,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso
0,0,5004523,2024-01-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
1,1,5004523,2024-02-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
2,2,5004523,2024-03-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
3,3,5004523,2024-04-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
4,4,5004523,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.15,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10497,5,1010166177_2,2024-01-01,2.33,VentaFacturadaPortafolioZonaClusterSub,3.777647e+07,16190154.54,0.0,0.0,0.0,...,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01
10498,6,1010166177_2,2024-02-01,7.98,VentaFacturadaPortafolioZonaClusterSub,2.583843e+08,32380309.08,0.0,0.0,0.0,...,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01
10499,7,1010166177_2,2024-03-01,30.12,VentaFacturadaPortafolioZonaClusterSub,1.544331e+09,51268822.71,0.0,0.0,0.0,...,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01
10500,8,1010166177_2,2024-04-01,23.10,VentaFacturadaPortafolioZonaClusterSub,1.620885e+09,70157336.34,0.0,0.0,0.0,...,0.15,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01


In [42]:
df_salarios_variables[df_salarios_variables['CodigoEmpleado'] == '16659046']

,CodigoEmpleado,Fecha,SalarioVariable,PromedioSalarioVariable
536,16659046,2024-01-01,1712000.0,1712000.0
537,16659046,2024-02-01,1712000.0,1712000.0
538,16659046,2024-03-01,1712000.0,1712000.0
539,16659046,2024-04-01,3149000.0,2071250.0
540,16659046,2024-05-01,3149000.0,2286800.0


In [43]:
maestra_resultados_por_empleado[maestra_resultados_por_empleado['CodigoEmpleado'] == '16659046']

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,_merge,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso
550,550,16659046,2024-01-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.2,0.0,Objetivos cualitativos,NaN,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17
551,551,16659046,2024-02-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.2,0.0,Objetivos cualitativos,NaN,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17
552,552,16659046,2024-03-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.2,0.0,Objetivos cualitativos,NaN,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17
553,553,16659046,2024-04-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.2,0.0,Objetivos cualitativos,NaN,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17
554,554,16659046,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.2,0.0,Objetivos cualitativos,NaN,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17
10149,60,16659046_1,2024-03-01,1.05,VentaFacturadaBraunPaisSUB,1.120332e+07,1.062285e+07,0.0,0.0,0.0,...,0.4,1.0,VentaFacturadaBraunPaisSUB,BBMCO,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17
10150,61,16659046_1,2024-04-01,1.09,VentaFacturadaBraunPaisSUB,1.547450e+07,1.420999e+07,0.0,0.0,0.0,...,0.4,1.0,VentaFacturadaBraunPaisSUB,BBMCO,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17
10151,62,16659046_1,2024-05-01,1.06,VentaFacturadaBraunPaisSUB,1.964573e+07,1.858577e+07,0.0,0.0,0.0,...,0.4,1.0,VentaFacturadaBraunPaisSUB,BBMCO,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17
10416,18,16659046_1,2024-03-01,1.18,EBITCOGSPaisSUB,2.980374e+06,2.534506e+06,0.0,0.0,0.0,...,0.4,1.0,EBITCOGSPaisSUB,BBMCO,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17
10417,19,16659046_1,2024-04-01,1.12,EBITCOGSPaisSUB,3.789609e+06,3.390361e+06,0.0,0.0,0.0,...,0.4,1.0,EBITCOGSPaisSUB,BBMCO,both,AD,3.0,REYES MONROY,ADOLFO MARIO,1994-05-17


In [44]:
# TODO Hacer el merge con la tabla de promedio variable
maestra_resultados_por_empleado = maestra_resultados_por_empleado.merge(
    df_salarios_variables[['CodigoEmpleado', 'Fecha', 'PromedioSalarioVariable']].rename(
        columns={'PromedioSalarioVariable': 'SalarioVariable'}
    ),
    on=['CodigoEmpleado', 'Fecha'],
    how='left'
)
maestra_resultados_por_empleado

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,Consecutivo,Nombre variable,Nombre consecutivo,_merge,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso,SalarioVariable
0,0,5004523,2024-01-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01,2500000.0
1,1,5004523,2024-02-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01,2500000.0
2,2,5004523,2024-03-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01,2500000.0
3,3,5004523,2024-04-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01,2500000.0
4,4,5004523,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.0,Objetivos cualitativos,NaN,both,VC,3.0,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01,2500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10471,5,1010166177_2,2024-01-01,2.33,VentaFacturadaPortafolioZonaClusterSub,3.777647e+07,16190154.54,0.0,0.0,0.0,...,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01,2500000.0
10472,6,1010166177_2,2024-02-01,7.98,VentaFacturadaPortafolioZonaClusterSub,2.583843e+08,32380309.08,0.0,0.0,0.0,...,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01,2500000.0
10473,7,1010166177_2,2024-03-01,30.12,VentaFacturadaPortafolioZonaClusterSub,1.544331e+09,51268822.71,0.0,0.0,0.0,...,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01,2500000.0
10474,8,1010166177_2,2024-04-01,23.10,VentaFacturadaPortafolioZonaClusterSub,1.620885e+09,70157336.34,0.0,0.0,0.0,...,2.0,VentaFacturadaPortafolioZonaClusterSub,Ambulatorio (Cluster 20-21),both,VE,9.0,Castiblanco Martinez,William Andres,2023-12-01,2500000.0


In [45]:
maestra_resultados_por_empleado[
     (maestra_resultados_por_empleado['Fecha']=='2024-05-01') &
    (maestra_resultados_por_empleado['Contexto']=='1111117')
]

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,Consecutivo,Nombre variable,Nombre consecutivo,_merge,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso,SalarioVariable
514,514,1111117,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.0,Objetivos cualitativos,NaN,both,VC,5.0,Pedraza,Julieth,2023-01-01,2500000.0
724,24,1111117,2024-05-01,0.88,VentaFacturadaZonaCargoIndividual,3.179236e+08,3.617851e+08,0.0,0.0,0.0,...,NaN,NaN,NaN,left_only,NaN,NaN,NaN,NaN,NaT,NaN
2054,24,1111117,2024-05-01,0.79,VentasCobradaZonaCargoIndividual,3.044938e+08,3.835532e+08,0.0,0.0,0.0,...,NaN,NaN,NaN,left_only,NaN,NaN,NaN,NaN,NaT,NaN
5389,29,1111117,2024-05-01,0.86,VentaCobradaBraunCluster,3.632880e+08,4.212271e+08,0.0,0.0,0.0,...,NaN,NaN,NaN,left_only,NaN,NaN,NaN,NaN,NaT,NaN
6482,24,1111117,2024-05-01,0.88,VentaFacturadaPortafolioCluster,3.179236e+08,3.617851e+08,0.0,0.0,0.0,...,NaN,NaN,NaN,left_only,NaN,NaN,NaN,NaN,NaT,NaN
7946,39,1111117,2024-05-01,0.94,VentaFacturadaZonaPais,5.855320e+10,6.258874e+10,0.0,0.0,0.0,...,NaN,NaN,NaN,left_only,NaN,NaN,NaN,NaN,NaT,NaN
8266,4,1111117,2024-05-01,3.37,VentaFacturadaPortafolioRenal,1.814526e+09,5.383562e+08,0.0,0.0,0.0,...,NaN,NaN,NaN,left_only,NaN,NaN,NaN,NaN,NaT,NaN
8825,24,1111117,2024-05-01,0.86,VentaCobradaPortafolioCluster,3.632880e+08,4.212271e+08,0.0,0.0,0.0,...,NaN,NaN,NaN,left_only,NaN,NaN,NaN,NaN,NaT,NaN
10460,4,1111117,2024-05-01,1.04,VentaCobradaPortafolioRenal,8.571691e+08,8.276971e+08,0.0,0.0,0.0,...,0.0,VentaCobradaPortafolioRenal,NaN,both,VC,5.0,Pedraza,Julieth,2023-01-01,2500000.0


### Maestra de Resultados por Empleado y Centro de Costos

In [46]:
maestra_resultados_centro_costo.head()

,Unnamed: 0,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,AreaCalculo,Consecutivo,TipoEmpleado,PorcentajeCumplimiento,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP
0,0,5002711_1,2024-01-01,610,HC,323015040,2.838878e+08,2.926409e+08,VentaCobradaZonaPaisSub,0,NaN,NaN,0.97,0.0,0.0,0.0
1,1,5002711_1,2024-01-01,620,HC,323015045,1.906658e+09,2.109654e+09,VentaCobradaZonaPaisSub,0,NaN,NaN,0.90,0.0,0.0,0.0
2,2,5002711_1,2024-01-01,630,HC,323015050,1.321169e+09,1.181046e+09,VentaCobradaZonaPaisSub,0,NaN,NaN,1.12,0.0,0.0,0.0
3,3,5002711_1,2024-01-01,640,HC,323015055,0.000000e+00,2.040000e+06,VentaCobradaZonaPaisSub,0,NaN,NaN,0.00,0.0,0.0,0.0
4,4,5002711_1,2024-01-01,680,HC,323015055,2.727908e+08,2.324993e+08,VentaCobradaZonaPaisSub,0,NaN,NaN,1.17,0.0,0.0,0.0


In [47]:
df_parrillas

,CodigoEmpleado,Variable,Contexto,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo
0,5004523,VentaCobradaBraunPais,Pais,0.300,0,Venta cobrada país,NaN
1,5004523,ObjetivosCualitativos,5004523,0.150,0,Objetivos cualitativos,NaN
2,5004523,VentaFacturadaPortafolioClusterSub,5004523_1,0.060,1,VentaFacturadaPortafolioClusterSub,Surgery & Interventions (Neuro)
3,5004523,VentaFacturadaPortafolioClusterSub,5004523_2,0.060,2,VentaFacturadaPortafolioClusterSub,Surgery & Interventions (Ortho)
4,5004523,VentaFacturadaPortafolioClusterSub,5004523_3,0.060,3,VentaFacturadaPortafolioClusterSub,Surgery & Interventions (Surgical Assets)
...,...,...,...,...,...,...,...
1342,5036653,VentaFacturadaPortafolioClusterSub,5036653_2,0.045,2,VentaFacturadaPortafolioClusterSub,Heridas.Ostomia y desinfeccion (ambulatorio)
1343,5036653,VentaFacturadaPortafolioClusterSub,5036653_3,0.075,3,VentaFacturadaPortafolioClusterSub,"Hospital Ambulatorio: Nutricion (640-680), Sis..."
1344,5036653,VentaFacturadaPortafolioClusterSub,5036653_4,0.030,4,VentaFacturadaPortafolioClusterSub,Aesculap (ambulatorio)
1345,5036653,VentaFacturadaPortafolioRenalSub,5036653_1,0.150,1,VentaFacturadaPortafolioRenalSub,Renal Total Nacional (hospitalio + ambulatorio...


In [48]:
maestra_resultados_por_empleado_centro_costo = maestra_resultados_centro_costo.drop(columns={'AreaCalculo', 'Consecutivo', 'TipoEmpleado', 'Unnamed: 0'})

In [49]:
maestra_resultados_por_empleado_centro_costo = maestra_resultados_por_empleado_centro_costo.merge(
    df_parrillas,
    on=['Contexto', 'Variable'],
    how='left'
)
maestra_resultados_por_empleado_centro_costo.head()

,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,PorcentajeCumplimiento,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,CodigoEmpleado,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo
0,5002711_1,2024-01-01,610,HC,323015040,2.838878e+08,2.926409e+08,VentaCobradaZonaPaisSub,0.97,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,5002711_1,2024-01-01,620,HC,323015045,1.906658e+09,2.109654e+09,VentaCobradaZonaPaisSub,0.90,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,5002711_1,2024-01-01,630,HC,323015050,1.321169e+09,1.181046e+09,VentaCobradaZonaPaisSub,1.12,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,5002711_1,2024-01-01,640,HC,323015055,0.000000e+00,2.040000e+06,VentaCobradaZonaPaisSub,0.00,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,5002711_1,2024-01-01,680,HC,323015055,2.727908e+08,2.324993e+08,VentaCobradaZonaPaisSub,1.17,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [50]:
maestra_resultados_por_empleado_centro_costo[
    (maestra_resultados_por_empleado_centro_costo['CodigoEmpleado'] == '5002899')
]   

,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,PorcentajeCumplimiento,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,CodigoEmpleado,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo


In [51]:
maestra_resultados_por_empleado_centro_costo[
    (maestra_resultados_por_empleado_centro_costo['CodigoEmpleado'] == '1032450706') &
    (maestra_resultados_por_empleado_centro_costo['Fecha'] == '2023-01-01')
]

,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,PorcentajeCumplimiento,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,CodigoEmpleado,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo


In [52]:
# maestra_resultados_por_empleado_centro_costo = maestra_resultados_centro_costo.drop(columns=['Unnamed: 0' , 'Consecutivo']).merge(
#     df_parrillas_tipos_calculos,
#     on=['TipoEmpleado', 'AreaCalculo'],
#     how='left'
# )
# maestra_resultados_por_empleado_centro_costo.head()

In [53]:
# maestra_resultados_por_empleado_centro_costo = maestra_resultados_por_empleado_centro_costo.merge(
#     df_empleados_centros_costos.rename(columns={'PorcentajeAsignacion': 'Porcentaje'})
# )

In [54]:
# maestra_resultados_por_empleado_centro_costo[maestra_resultados_por_empleado_centro_costo['CodigoEmpleado'] == '1032450706']

In [55]:
df_empleados[['CodigoEmpleado', 'TipoEmpleado', 'AreaCalculo', 'Apellidos', 'Nombre', 'FechaIngreso', 'FechaRetiro']][df_empleados['CodigoEmpleado'] == '5005463']

,CodigoEmpleado,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso,FechaRetiro


In [56]:
df_empleados[['CodigoEmpleado', 'TipoEmpleado', 'AreaCalculo', 'Apellidos', 'Nombre', 'FechaIngreso', 'FechaRetiro']][df_empleados['CodigoEmpleado'] == '1098612757']

,CodigoEmpleado,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso,FechaRetiro
126,1098612757,AD1,3,Serrano,Andres,2023-06-01,NaT


In [57]:
df_empleados_inactivos[['CodigoEmpleado', 'TipoEmpleado', 'AreaCalculo', 'Apellidos', 'Nombre', 'FechaIngreso', 'FechaRetiro']][df_empleados_inactivos['CodigoEmpleado'] == '1032450706']

,CodigoEmpleado,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso,FechaRetiro
81,1032450706,MK,9,RODRIGUEZ CASTELLANOS,LAURA CAMILA,2022-07-01,NaT


In [58]:
df_empleados_inactivos[['CodigoEmpleado', 'TipoEmpleado', 'AreaCalculo', 'Apellidos', 'Nombre', 'FechaIngreso', 'FechaRetiro']][df_empleados_inactivos['CodigoEmpleado'] == '5005463']

,CodigoEmpleado,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso,FechaRetiro
19,5005463,GU,6,CHAVES,DIANA,2016-07-01,NaT


In [59]:
df_empleados_totales = pd.concat([df_empleados, df_empleados_inactivos])

In [60]:
maestra_resultados_por_empleado_centro_costo = maestra_resultados_por_empleado_centro_costo.merge(
    df_empleados_totales[['CodigoEmpleado', 'TipoEmpleado', 'AreaCalculo', 'Apellidos', 'Nombre', 'FechaIngreso']],
    on=['CodigoEmpleado'],
    how='left'
)
maestra_resultados_por_empleado_centro_costo.head()

,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,PorcentajeCumplimiento,RealCOP,...,CodigoEmpleado,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso
0,5002711_1,2024-01-01,610,HC,323015040,2.838878e+08,2.926409e+08,VentaCobradaZonaPaisSub,0.97,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,5002711_1,2024-01-01,620,HC,323015045,1.906658e+09,2.109654e+09,VentaCobradaZonaPaisSub,0.90,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,5002711_1,2024-01-01,630,HC,323015050,1.321169e+09,1.181046e+09,VentaCobradaZonaPaisSub,1.12,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,5002711_1,2024-01-01,640,HC,323015055,0.000000e+00,2.040000e+06,VentaCobradaZonaPaisSub,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,5002711_1,2024-01-01,680,HC,323015055,2.727908e+08,2.324993e+08,VentaCobradaZonaPaisSub,1.17,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [61]:
# Se añaden líneas para filtrar registros vacíos ('Fecha' y 'CodigoEmpleado')
maestra_resultados_por_empleado_centro_costo.dropna(subset=[
        'Fecha'
    ], inplace=True)
maestra_resultados_por_empleado_centro_costo['CodigoEmpleado'] = maestra_resultados_por_empleado_centro_costo['CodigoEmpleado'].astype(str)
maestra_resultados_por_empleado_centro_costo.head()

,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,PorcentajeCumplimiento,RealCOP,...,CodigoEmpleado,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso
0,5002711_1,2024-01-01,610,HC,323015040,2.838878e+08,2.926409e+08,VentaCobradaZonaPaisSub,0.97,0.0,...,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,5002711_1,2024-01-01,620,HC,323015045,1.906658e+09,2.109654e+09,VentaCobradaZonaPaisSub,0.90,0.0,...,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,5002711_1,2024-01-01,630,HC,323015050,1.321169e+09,1.181046e+09,VentaCobradaZonaPaisSub,1.12,0.0,...,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,5002711_1,2024-01-01,640,HC,323015055,0.000000e+00,2.040000e+06,VentaCobradaZonaPaisSub,0.00,0.0,...,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,5002711_1,2024-01-01,680,HC,323015055,2.727908e+08,2.324993e+08,VentaCobradaZonaPaisSub,1.17,0.0,...,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [62]:
# TODO Hacer el merge con la tabla de promedio variable
maestra_resultados_por_empleado_centro_costo = maestra_resultados_por_empleado_centro_costo.merge(
    df_salarios_variables[['CodigoEmpleado', 'Fecha', 'PromedioSalarioVariable']].rename(
        columns={'PromedioSalarioVariable': 'SalarioVariable'}
    ),
    on=['CodigoEmpleado', 'Fecha'],
    how='left'
)
maestra_resultados_por_empleado_centro_costo.head()

,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,PorcentajeCumplimiento,RealCOP,...,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso,SalarioVariable
0,5002711_1,2024-01-01,610,HC,323015040,2.838878e+08,2.926409e+08,VentaCobradaZonaPaisSub,0.97,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,5002711_1,2024-01-01,620,HC,323015045,1.906658e+09,2.109654e+09,VentaCobradaZonaPaisSub,0.90,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,5002711_1,2024-01-01,630,HC,323015050,1.321169e+09,1.181046e+09,VentaCobradaZonaPaisSub,1.12,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,5002711_1,2024-01-01,640,HC,323015055,0.000000e+00,2.040000e+06,VentaCobradaZonaPaisSub,0.00,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,5002711_1,2024-01-01,680,HC,323015055,2.727908e+08,2.324993e+08,VentaCobradaZonaPaisSub,1.17,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [63]:
maestra_resultados_por_empleado[
     (maestra_resultados_por_empleado['Fecha']=='2024-05-01') &
    (maestra_resultados_por_empleado['Contexto']=='1111117') &
    (maestra_resultados_por_empleado['Variable']=='ObjetivosCualitativos')
]

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,Consecutivo,Nombre variable,Nombre consecutivo,_merge,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso,SalarioVariable
514,514,1111117,2024-05-01,0.8,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,0.0,Objetivos cualitativos,NaN,both,VC,5.0,Pedraza,Julieth,2023-01-01,2500000.0


In [64]:
maestra_resultados_por_empleado_centro_costo[maestra_resultados_por_empleado_centro_costo['CodigoEmpleado'] == '1032450706']

,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,PorcentajeCumplimiento,RealCOP,...,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso,SalarioVariable


### Calculo sobre maestra resultados por empleado

In [65]:
maestra_resultados_por_empleado = maestra_resultados_por_empleado.merge(
    df_factores_liquidacion,
    on=['TipoEmpleado', 'PorcentajeCumplimiento'],
    how='left'
)

maestra_resultados_por_empleado['FactorIncentivo'] = np.where(
    ((maestra_resultados_por_empleado['Fecha'] >= '2022-08-01') & 
     (maestra_resultados_por_empleado['PorcentajeCumplimiento'] == 0) & 
     (maestra_resultados_por_empleado['Presupuesto'] == 0) & 
     (maestra_resultados_por_empleado['Real'] == 0)),
    0.25,
    maestra_resultados_por_empleado['FactorIncentivo']
)
maestra_resultados_por_empleado['FactorIncentivo'] = np.where(
    ((maestra_resultados_por_empleado['Fecha'] >= '2022-08-01') & 
     (maestra_resultados_por_empleado['PorcentajeCumplimiento'] == 0) & 
     (maestra_resultados_por_empleado['Presupuesto'] == 0) & 
     (maestra_resultados_por_empleado['Real'] > 0)),
    0.5,
    maestra_resultados_por_empleado['FactorIncentivo']
)

##
_excepciones_porcentaje_cumplimiento = [
    # {
    #     'TipoEmpleado': 'VC',
    #     'AreaCalculo': 7,
    #     'Variable': 'VentaFacturadaZonaCargoIndividualSub',
    #     'Consecutivo': 4,
    #     'FechaInicial': '2021-02-01',
    #     'FechaFinal': '2021-06-01',
    #     'PorcentajeCumplimiento': 1,
    #     'FactorIncentivo': 1,
    #     'Codigos': ['5003527']
    # }
]

for excepcion in _excepciones_porcentaje_cumplimiento:
    maestra_resultados_por_empleado['PorcentajeCumplimiento'] = np.where(
        (
            (maestra_resultados_por_empleado['CodigoEmpleado'].isin(excepcion['Codigos'])) &
            (maestra_resultados_por_empleado['Consecutivo'] == excepcion['Consecutivo']) &
            (maestra_resultados_por_empleado['Variable'] == excepcion['Variable']) &
            (maestra_resultados_por_empleado['Fecha'] >= excepcion['FechaInicial']) &
            (maestra_resultados_por_empleado['Fecha'] <= excepcion['FechaFinal'])
        ),
        excepcion['PorcentajeCumplimiento'],
        maestra_resultados_por_empleado['PorcentajeCumplimiento']
    )
    maestra_resultados_por_empleado['FactorIncentivo'] = np.where(
        (
            (maestra_resultados_por_empleado['CodigoEmpleado'].isin(excepcion['Codigos'])) &
            (maestra_resultados_por_empleado['Consecutivo'] == excepcion['Consecutivo']) &
            (maestra_resultados_por_empleado['Variable'] == excepcion['Variable']) &
            (maestra_resultados_por_empleado['Fecha'] >= excepcion['FechaInicial']) &
            (maestra_resultados_por_empleado['Fecha'] <= excepcion['FechaFinal'])
        ),
        excepcion['FactorIncentivo'],
        maestra_resultados_por_empleado['FactorIncentivo']
    )
##

maestra_resultados_por_empleado['FactorIncentivo'] = np.where(
    (maestra_resultados_por_empleado['PorcentajeCumplimiento']<0.9) & (maestra_resultados_por_empleado['FactorIncentivo'].isnull()),
    0,
    maestra_resultados_por_empleado['FactorIncentivo']
)
maestra_resultados_por_empleado['FactorIncentivo'] = np.where(
    (maestra_resultados_por_empleado['PorcentajeCumplimiento']>1.06) & (maestra_resultados_por_empleado['FactorIncentivo'].isnull()),
    2.5,
    maestra_resultados_por_empleado['FactorIncentivo']
)
maestra_resultados_por_empleado = maestra_resultados_por_empleado[maestra_resultados_por_empleado['Fecha'].notnull()]
maestra_resultados_por_empleado['FactorMes'] = pd.DatetimeIndex(maestra_resultados_por_empleado['Fecha']).month
maestra_resultados_por_empleado['FactorMes'] = maestra_resultados_por_empleado['FactorMes'].astype('int')
maestra_resultados_por_empleado['Liquidado'] = maestra_resultados_por_empleado['SalarioVariable']*maestra_resultados_por_empleado['Porcentaje']
maestra_resultados_por_empleado['Liquidado'] = maestra_resultados_por_empleado['Liquidado']*maestra_resultados_por_empleado['FactorIncentivo']

try:
    maestra_resultados_por_empleado['DifMonths'] = ((maestra_resultados_por_empleado.Fecha - maestra_resultados_por_empleado.FechaIngreso)/np.timedelta64(1, 'M'))
except:
    maestra_resultados_por_empleado['DifMonths'] = 0
maestra_resultados_por_empleado['DifMonths'].fillna(0, inplace=True)
maestra_resultados_por_empleado['DifMonths'] = maestra_resultados_por_empleado['DifMonths'].apply(lambda x: round(x, 0))
maestra_resultados_por_empleado['DifMonths'] = maestra_resultados_por_empleado['DifMonths'].astype('int')

maestra_resultados_por_empleado['DifMonths'] = maestra_resultados_por_empleado.groupby(['CodigoEmpleado'])['DifMonths'].transform(min)
maestra_resultados_por_empleado['DifMonths'] = np.where(
    maestra_resultados_por_empleado['DifMonths']>0, 0, maestra_resultados_por_empleado['DifMonths']
)
maestra_resultados_por_empleado['NuevoFactor'] = maestra_resultados_por_empleado['FactorMes'] + maestra_resultados_por_empleado['DifMonths']
maestra_resultados_por_empleado['NuevoFactor'] = maestra_resultados_por_empleado['NuevoFactor'].apply(lambda x: 0 if x<0 else x)

###
_excepciones_factor_mes = [
    # Garantizados para VEs y VCs
    {
        'TipoEmpleado': ['VC', 'VE'],
        'FechaInicial': '2022-03-01',
        'FechaFinal': '2022-12-01',
        'SumFactorMes': -2
    }
]
for excepcion in _excepciones_factor_mes:
    maestra_resultados_por_empleado['NuevoFactor'] = np.where(
        (
            (maestra_resultados_por_empleado['TipoEmpleado'].isin(excepcion['TipoEmpleado'])) &
            (maestra_resultados_por_empleado['Fecha'] >= excepcion['FechaInicial']) &
            (maestra_resultados_por_empleado['Fecha'] <= excepcion['FechaFinal'])
        ),
        maestra_resultados_por_empleado['NuevoFactor'] + excepcion['SumFactorMes'],
        maestra_resultados_por_empleado['NuevoFactor']
    )

maestra_resultados_por_empleado['NuevoFactor'] = np.where(
    maestra_resultados_por_empleado['NuevoFactor'] < 1,
    1,
    maestra_resultados_por_empleado['NuevoFactor']
)
maestra_resultados_por_empleado['NuevoFactorBK'] = maestra_resultados_por_empleado['NuevoFactor']
###

maestra_resultados_por_empleado['Liquidado'] = maestra_resultados_por_empleado['Liquidado']*maestra_resultados_por_empleado['NuevoFactor']

maestra_resultados_por_empleado = maestra_resultados_por_empleado[maestra_resultados_por_empleado['CodigoEmpleado'].notnull()]

C:\Users\jjgg\AppData\Local\Temp\ipykernel_25464\2488020511.py:88: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  maestra_resultados_por_empleado['DifMonths'] = maestra_resultados_por_empleado.groupby(['CodigoEmpleado'])['DifMonths'].transform(min)


In [66]:
maestra_resultados_por_empleado

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,Apellidos,Nombre,FechaIngreso,SalarioVariable,FactorIncentivo,FactorMes,Liquidado,DifMonths,NuevoFactor,NuevoFactorBK
0,0,5004523,2024-01-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01,2500000.0,0.5,1,187500.0,0,1,1
1,1,5004523,2024-02-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01,2500000.0,0.5,2,375000.0,0,2,2
2,2,5004523,2024-03-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01,2500000.0,0.5,3,562500.0,0,3,3
3,3,5004523,2024-04-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01,2500000.0,0.5,4,750000.0,0,4,4
4,4,5004523,2024-05-01,0.80,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,ACOSTA PORTILLO,SOBEYDA DEL SOCORRO,2023-10-01,2500000.0,0.5,5,937500.0,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10471,5,1010166177_2,2024-01-01,2.33,VentaFacturadaPortafolioZonaClusterSub,3.777647e+07,16190154.54,0.0,0.0,0.0,...,Castiblanco Martinez,William Andres,2023-12-01,2500000.0,2.5,1,937500.0,0,1,1
10472,6,1010166177_2,2024-02-01,7.98,VentaFacturadaPortafolioZonaClusterSub,2.583843e+08,32380309.08,0.0,0.0,0.0,...,Castiblanco Martinez,William Andres,2023-12-01,2500000.0,2.5,2,1875000.0,0,2,2
10473,7,1010166177_2,2024-03-01,30.12,VentaFacturadaPortafolioZonaClusterSub,1.544331e+09,51268822.71,0.0,0.0,0.0,...,Castiblanco Martinez,William Andres,2023-12-01,2500000.0,2.5,3,2812500.0,0,3,3
10474,8,1010166177_2,2024-04-01,23.10,VentaFacturadaPortafolioZonaClusterSub,1.620885e+09,70157336.34,0.0,0.0,0.0,...,Castiblanco Martinez,William Andres,2023-12-01,2500000.0,2.5,4,3750000.0,0,4,4


In [67]:
maestra_resultados_por_empleado[
     (maestra_resultados_por_empleado['Fecha']=='2024-05-01') &
    (maestra_resultados_por_empleado['Contexto']=='1111117') &
    (maestra_resultados_por_empleado['Variable']=='ObjetivosCualitativos')
]

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,Apellidos,Nombre,FechaIngreso,SalarioVariable,FactorIncentivo,FactorMes,Liquidado,DifMonths,NuevoFactor,NuevoFactorBK
514,514,1111117,2024-05-01,0.8,ObjetivosCualitativos,NaN,NaN,0.0,0.0,0.0,...,Pedraza,Julieth,2023-01-01,2500000.0,0.5,5,937500.0,0,5,5


### Calculo sobre maestra resultados por empleado centro costo

In [68]:
maestra_resultados_por_empleado_centro_costo_final = maestra_resultados_por_empleado_centro_costo.merge(
    df_factores_liquidacion,
    on=['TipoEmpleado', 'PorcentajeCumplimiento'],
    how='left'
)

maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo'] = np.where(
    ((maestra_resultados_por_empleado_centro_costo_final['Fecha'] >= '2022-08-01') & 
     (maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimiento'] == 0) & 
     (maestra_resultados_por_empleado_centro_costo_final['Presupuesto'] == 0) & 
     (maestra_resultados_por_empleado_centro_costo_final['Real'] == 0)),
    0.25,
    maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo']
)
maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo'] = np.where(
    ((maestra_resultados_por_empleado_centro_costo_final['Fecha'] >= '2022-08-01') & 
     (maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimiento'] == 0) & 
     (maestra_resultados_por_empleado_centro_costo_final['Presupuesto'] == 0) & 
     (maestra_resultados_por_empleado_centro_costo_final['Real'] > 0)),
    0.5,
    maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo']
)

##
_excepciones_porcentaje_cumplimiento = [
    # {
    #     'TipoEmpleado': 'VC',
    #     'AreaCalculo': 7,
    #     'Variable': 'VentaFacturadaZonaCargoIndividualSub',
    #     'Consecutivo': 4,
    #     'FechaInicial': '2021-02-01',
    #     'FechaFinal': '2021-06-01',
    #     'PorcentajeCumplimiento': 1,
    #     'FactorIncentivo': 1,
    #     'Codigos': ['5003527']
    # }
]

for excepcion in _excepciones_porcentaje_cumplimiento:
    maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimiento'] = np.where(
        (
            (maestra_resultados_por_empleado_centro_costo_final['CodigoEmpleado'].isin(excepcion['Codigos'])) &
            (maestra_resultados_por_empleado_centro_costo_final['Consecutivo'] == excepcion['Consecutivo']) &
            (maestra_resultados_por_empleado_centro_costo_final['Variable'] == excepcion['Variable']) &
            (maestra_resultados_por_empleado_centro_costo_final['Fecha'] >= excepcion['FechaInicial']) &
            (maestra_resultados_por_empleado_centro_costo_final['Fecha'] <= excepcion['FechaFinal'])
        ),
        excepcion['PorcentajeCumplimiento'],
        maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimiento']
    )
    maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo'] = np.where(
        (
            (maestra_resultados_por_empleado_centro_costo_final['CodigoEmpleado'].isin(excepcion['Codigos'])) &
            (maestra_resultados_por_empleado_centro_costo_final['Consecutivo'] == excepcion['Consecutivo']) &
            (maestra_resultados_por_empleado_centro_costo_final['Variable'] == excepcion['Variable']) &
            (maestra_resultados_por_empleado_centro_costo_final['Fecha'] >= excepcion['FechaInicial']) &
            (maestra_resultados_por_empleado_centro_costo_final['Fecha'] <= excepcion['FechaFinal'])
        ),
        excepcion['FactorIncentivo'],
        maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo']
    )
##

maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo'] = np.where(
    (maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimiento']<0.9) & (maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo'].isnull()),
    0,
    maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo']
)
maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo'] = np.where(
    (maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimiento']>1.06) & (maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo'].isnull()),
    2.5,
    maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo']
)
maestra_resultados_por_empleado_centro_costo_final = maestra_resultados_por_empleado_centro_costo_final[maestra_resultados_por_empleado_centro_costo_final['Fecha'].notnull()]
maestra_resultados_por_empleado_centro_costo_final['FactorMes'] = pd.DatetimeIndex(maestra_resultados_por_empleado_centro_costo_final['Fecha']).month
maestra_resultados_por_empleado_centro_costo_final['FactorMes'] = maestra_resultados_por_empleado_centro_costo_final['FactorMes'].astype('int')
maestra_resultados_por_empleado_centro_costo_final['Liquidado'] = maestra_resultados_por_empleado_centro_costo_final['SalarioVariable']*maestra_resultados_por_empleado_centro_costo_final['Porcentaje']
maestra_resultados_por_empleado_centro_costo_final['Liquidado'] = maestra_resultados_por_empleado_centro_costo_final['Liquidado']*maestra_resultados_por_empleado_centro_costo_final['FactorIncentivo']

try:
    maestra_resultados_por_empleado_centro_costo_final['DifMonths'] = ((maestra_resultados_por_empleado_centro_costo_final.Fecha - maestra_resultados_por_empleado_centro_costo_final.FechaIngreso)/np.timedelta64(1, 'M'))
except:
    maestra_resultados_por_empleado_centro_costo_final['DifMonths'] = 0
maestra_resultados_por_empleado_centro_costo_final['DifMonths'].fillna(0, inplace=True)
maestra_resultados_por_empleado_centro_costo_final['DifMonths'] = maestra_resultados_por_empleado_centro_costo_final['DifMonths'].apply(lambda x: round(x, 0))
maestra_resultados_por_empleado_centro_costo_final['DifMonths'] = maestra_resultados_por_empleado_centro_costo_final['DifMonths'].astype('int')

maestra_resultados_por_empleado_centro_costo_final['DifMonths'] = maestra_resultados_por_empleado_centro_costo_final.groupby(['CodigoEmpleado'])['DifMonths'].transform(min)
maestra_resultados_por_empleado_centro_costo_final['DifMonths'] = np.where(
    maestra_resultados_por_empleado_centro_costo_final['DifMonths']>0, 0, maestra_resultados_por_empleado_centro_costo_final['DifMonths']
)
maestra_resultados_por_empleado_centro_costo_final['NuevoFactor'] = maestra_resultados_por_empleado_centro_costo_final['FactorMes'] + maestra_resultados_por_empleado_centro_costo_final['DifMonths']
maestra_resultados_por_empleado_centro_costo_final['NuevoFactor'] = maestra_resultados_por_empleado_centro_costo_final['NuevoFactor'].apply(lambda x: 0 if x<0 else x)

###
_excepciones_factor_mes = [
    # Garantizados para VEs y VCs
    {
        'TipoEmpleado': ['VC', 'VE'],
        'FechaInicial': '2022-03-01',
        'FechaFinal': '2022-12-01',
        'SumFactorMes': -2
    }
]
for excepcion in _excepciones_factor_mes:
    maestra_resultados_por_empleado_centro_costo_final['NuevoFactor'] = np.where(
        (
            (maestra_resultados_por_empleado_centro_costo_final['TipoEmpleado'].isin(excepcion['TipoEmpleado'])) &
            (maestra_resultados_por_empleado_centro_costo_final['Fecha'] >= excepcion['FechaInicial']) &
            (maestra_resultados_por_empleado_centro_costo_final['Fecha'] <= excepcion['FechaFinal'])
        ),
        maestra_resultados_por_empleado_centro_costo_final['NuevoFactor'] + excepcion['SumFactorMes'],
        maestra_resultados_por_empleado_centro_costo_final['NuevoFactor']
    )

maestra_resultados_por_empleado_centro_costo_final['NuevoFactor'] = np.where(
    maestra_resultados_por_empleado_centro_costo_final['NuevoFactor'] < 1,
    1,
    maestra_resultados_por_empleado_centro_costo_final['NuevoFactor']
)
maestra_resultados_por_empleado_centro_costo_final['NuevoFactorBK'] = maestra_resultados_por_empleado_centro_costo_final['NuevoFactor']
###

maestra_resultados_por_empleado_centro_costo_final['Liquidado'] = maestra_resultados_por_empleado_centro_costo_final['Liquidado']*maestra_resultados_por_empleado_centro_costo_final['NuevoFactor']

maestra_resultados_por_empleado_centro_costo_final = maestra_resultados_por_empleado_centro_costo_final[maestra_resultados_por_empleado_centro_costo_final['CodigoEmpleado'].notnull()]

C:\Users\jjgg\AppData\Local\Temp\ipykernel_25464\1614624925.py:88: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  maestra_resultados_por_empleado_centro_costo_final['DifMonths'] = maestra_resultados_por_empleado_centro_costo_final.groupby(['CodigoEmpleado'])['DifMonths'].transform(min)


In [69]:
maestra_resultados_por_empleado_centro_costo_final.head()

,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,PorcentajeCumplimiento,RealCOP,...,Apellidos,Nombre,FechaIngreso,SalarioVariable,FactorIncentivo,FactorMes,Liquidado,DifMonths,NuevoFactor,NuevoFactorBK
0,5002711_1,2024-01-01,610,HC,323015040,2.838878e+08,2.926409e+08,VentaCobradaZonaPaisSub,0.97,0.0,...,NaN,NaN,NaT,NaN,NaN,1,NaN,0,1,1
1,5002711_1,2024-01-01,620,HC,323015045,1.906658e+09,2.109654e+09,VentaCobradaZonaPaisSub,0.90,0.0,...,NaN,NaN,NaT,NaN,NaN,1,NaN,0,1,1
2,5002711_1,2024-01-01,630,HC,323015050,1.321169e+09,1.181046e+09,VentaCobradaZonaPaisSub,1.12,0.0,...,NaN,NaN,NaT,NaN,2.5,1,NaN,0,1,1
3,5002711_1,2024-01-01,640,HC,323015055,0.000000e+00,2.040000e+06,VentaCobradaZonaPaisSub,0.00,0.0,...,NaN,NaN,NaT,NaN,0.0,1,NaN,0,1,1
4,5002711_1,2024-01-01,680,HC,323015055,2.727908e+08,2.324993e+08,VentaCobradaZonaPaisSub,1.17,0.0,...,NaN,NaN,NaT,NaN,2.5,1,NaN,0,1,1


In [70]:
maestra_resultados_por_empleado_centro_costo_final[maestra_resultados_por_empleado_centro_costo_final['CodigoEmpleado'] == '1098612757']

,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,PorcentajeCumplimiento,RealCOP,...,Apellidos,Nombre,FechaIngreso,SalarioVariable,FactorIncentivo,FactorMes,Liquidado,DifMonths,NuevoFactor,NuevoFactorBK
11212,1098612757_1,2024-03-01,BBMCO,BBMCO,323004021,1.120332e+07,1.062285e+07,VentaFacturadaBraunPaisSUB,1.05,0.0,...,Serrano,Andres,2023-06-01,600000.0,2.25,3,1620000.0,0,3,3
11213,1098612757_1,2024-04-01,BBMCO,BBMCO,323004021,2.667782e+07,2.483285e+07,VentaFacturadaBraunPaisSUB,1.07,0.0,...,Serrano,Andres,2023-06-01,600000.0,2.50,4,2400000.0,0,4,4
11214,1098612757_1,2024-05-01,BBMCO,BBMCO,323004021,4.632354e+07,4.341862e+07,VentaFacturadaBraunPaisSUB,1.07,0.0,...,Serrano,Andres,2023-06-01,600000.0,2.50,5,3000000.0,0,5,5


## Se calculan los resultados por mes

In [71]:
lista_variables_rentabilidad_acumuladas = [
    'RentabilidadBbraunPaisAntesDeImpuestos',
    'RentabilidadDivisionGrupoClientesCM3',
    'RentabilidadDivisionGrupoClientesCM3Sub',
    'RentabilidadUnidadNegocioCM2',
    'RentabilidadUnidadNegocioCM2Sub',
    'RentabilidadUnidadNegocioCM3',
    'RentabilidadUnidadNegocioCM3Sub',
    'TotalRentabilidadBraunAntesDeImpuestosCM5',
    'VentaFacturadaPrevioNotasCreditoTP',
    # 2022
    'RentabilidadCompaniaCM3',
    'RentabilidadZonaHospitalarioCM2',
    # 2024
    'RentabilidadUnidadNegocioCM2COGSSUB',
    'RentabilidadDivisionCM2COGsSub',
    'EBITCOGSPais',
    'EBITCOGSPaisSUB'
]

def calcular_resultado_mes(group):
    valor_previo_real = 0
    valor_previo_presupuesto = 0
    for index, item in group.iterrows():
        if item['Variable'] in lista_variables_rentabilidad_acumuladas:
            resultado_real_mes = item['Real']
            resultado_presupuesto_mes = item['Presupuesto']
        else:
            resultado_real_mes = item['Real'] - valor_previo_real
            resultado_presupuesto_mes = item['Presupuesto'] - valor_previo_presupuesto
        
        group.at[index, 'ResultadoRealMes'] = resultado_real_mes
        group.at[index, 'ResultadoPresupuesto'] = resultado_presupuesto_mes

        porcentaje_cumplimiento_mes = 0
        if resultado_presupuesto_mes == 0:
            porcentaje_cumplimiento_mes = 0
        elif resultado_real_mes - resultado_presupuesto_mes < 0:
            porcentaje_cumplimiento_mes = 1 + ((resultado_real_mes - resultado_presupuesto_mes) / abs(resultado_presupuesto_mes))
        elif resultado_real_mes - resultado_presupuesto_mes > 0:
            porcentaje_cumplimiento_mes = 1 + abs((resultado_real_mes - resultado_presupuesto_mes) / abs(resultado_presupuesto_mes))
        else:
            porcentaje_cumplimiento_mes = resultado_real_mes / resultado_presupuesto_mes

        group.at[index, 'PorcentajeCumplimientoMes'] = round(porcentaje_cumplimiento_mes, 2)
        
        valor_previo_real = item['Real']
        valor_previo_presupuesto = item['Presupuesto']
    return group


### Maestra Resultados por Empleado vs Resultado Detallado Histórico

In [72]:
maestra_resultados_por_empleado['Real'].fillna(0,inplace=True)
maestra_resultados_por_empleado['Presupuesto'].fillna(0,inplace=True)
maestra_resultados_por_empleado['ResultadoPresupuesto'] = 0
maestra_resultados_por_empleado['ResultadoRealMes'] = 0
maestra_resultados_por_empleado['PorcentajeCumplimientoMes'] = 0
maestra_resultados_por_empleado['PorcentajeCumplimientoMes'] = maestra_resultados_por_empleado['PorcentajeCumplimientoMes'].astype('float')
maestra_resultados_por_empleado = maestra_resultados_por_empleado.groupby(['CodigoEmpleado', 'Variable', 'Contexto'], group_keys=False).apply(calcular_resultado_mes)

maestra_resultados_por_empleado

C:\Users\jjgg\AppData\Local\Temp\ipykernel_25464\3155329719.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2980374.20915347' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group.at[index, 'ResultadoRealMes'] = resultado_real_mes
C:\Users\jjgg\AppData\Local\Temp\ipykernel_25464\3155329719.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2534505.750457211' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group.at[index, 'ResultadoPresupuesto'] = resultado_presupuesto_mes
C:\Users\jjgg\AppData\Local\Temp\ipykernel_25464\3155329719.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2293197.425998528' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,SalarioVariable,FactorIncentivo,FactorMes,Liquidado,DifMonths,NuevoFactor,NuevoFactorBK,ResultadoPresupuesto,ResultadoRealMes,PorcentajeCumplimientoMes
0,0,5004523,2024-01-01,0.80,ObjetivosCualitativos,0.000000e+00,0.00,0.0,0.0,0.0,...,2500000.0,0.5,1,187500.0,0,1,1,0.00,0.000000e+00,0.00
1,1,5004523,2024-02-01,0.80,ObjetivosCualitativos,0.000000e+00,0.00,0.0,0.0,0.0,...,2500000.0,0.5,2,375000.0,0,2,2,0.00,0.000000e+00,0.00
2,2,5004523,2024-03-01,0.80,ObjetivosCualitativos,0.000000e+00,0.00,0.0,0.0,0.0,...,2500000.0,0.5,3,562500.0,0,3,3,0.00,0.000000e+00,0.00
3,3,5004523,2024-04-01,0.80,ObjetivosCualitativos,0.000000e+00,0.00,0.0,0.0,0.0,...,2500000.0,0.5,4,750000.0,0,4,4,0.00,0.000000e+00,0.00
4,4,5004523,2024-05-01,0.80,ObjetivosCualitativos,0.000000e+00,0.00,0.0,0.0,0.0,...,2500000.0,0.5,5,937500.0,0,5,5,0.00,0.000000e+00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10471,5,1010166177_2,2024-01-01,2.33,VentaFacturadaPortafolioZonaClusterSub,3.777647e+07,16190154.54,0.0,0.0,0.0,...,2500000.0,2.5,1,937500.0,0,1,1,16190154.54,3.777647e+07,2.33
10472,6,1010166177_2,2024-02-01,7.98,VentaFacturadaPortafolioZonaClusterSub,2.583843e+08,32380309.08,0.0,0.0,0.0,...,2500000.0,2.5,2,1875000.0,0,2,2,16190154.54,2.206078e+08,13.63
10473,7,1010166177_2,2024-03-01,30.12,VentaFacturadaPortafolioZonaClusterSub,1.544331e+09,51268822.71,0.0,0.0,0.0,...,2500000.0,2.5,3,2812500.0,0,3,3,18888513.63,1.285947e+09,68.08
10474,8,1010166177_2,2024-04-01,23.10,VentaFacturadaPortafolioZonaClusterSub,1.620885e+09,70157336.34,0.0,0.0,0.0,...,2500000.0,2.5,4,3750000.0,0,4,4,18888513.63,7.655384e+07,4.05


In [73]:
maestra_resultados_por_empleado[
     (maestra_resultados_por_empleado['Fecha']=='2024-05-01') &
    (maestra_resultados_por_empleado['Contexto']=='1111117') &
    (maestra_resultados_por_empleado['Variable']=='ObjetivosCualitativos')
]

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,SalarioVariable,FactorIncentivo,FactorMes,Liquidado,DifMonths,NuevoFactor,NuevoFactorBK,ResultadoPresupuesto,ResultadoRealMes,PorcentajeCumplimientoMes
514,514,1111117,2024-05-01,0.8,ObjetivosCualitativos,0.0,0.0,0.0,0.0,0.0,...,2500000.0,0.5,5,937500.0,0,5,5,0.0,0.0,0.0


In [74]:
df_resultado_detallado_previo

,Contexto,Fecha,Variable,PorcentajeCumplimiento,Real,Presupuesto,CodigoEmpleado,Porcentaje,Consecutivo,TipoEmpleado,...,Nombre,FechaIngreso,FactorIncentivo,FactorMes,Liquidado,DifMonths,NuevoFactor,ResultadoPresupuesto,ResultadoRealMes,PorcentajeCumplimientoMes
0,Pais,2024-02-01,VentaCobradaBraunPais,0.88,3.221882e+10,3.663783e+10,1010166177.0,0.300,0,VE,...,William Andres,2023-12-01,0.70,2,1050000.0,0,2,1.913977e+10,1.551444e+10,0.81
1,1010166177_1,2024-02-01,VentaFacturadaPortafolioZonaClusterSub,1.66,3.157478e+08,1.900081e+08,1010166177.0,0.150,1,VE,...,William Andres,2023-12-01,2.50,2,1875000.0,0,2,9.500404e+07,2.604769e+08,2.74
2,1010166177,2024-02-01,VentaCobradaPortafolioZonaCluster,2.89,3.886020e+08,1.345891e+08,1010166177.0,0.250,0,VE,...,William Andres,2023-12-01,2.50,2,3125000.0,0,2,6.729453e+07,2.759412e+08,4.10
3,Pais,2024-03-01,VentaCobradaBraunPais,0.86,4.803727e+10,5.604000e+10,1010166177.0,0.300,0,VE,...,William Andres,2023-12-01,0.65,3,1462500.0,0,3,1.940218e+10,1.581845e+10,0.82
4,Pais,2024-02-01,VentaCobradaBraunPais,0.88,3.221882e+10,3.663783e+10,1014204375.0,0.300,0,VE,...,Caterin Julieth,2024-01-01,0.70,2,1050000.0,0,2,1.913977e+10,1.551444e+10,0.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3114,9990011_1,2024-04-01,VentaFacturadaPortafolioClusterSub,0.79,7.307934e+08,9.299402e+08,9990011.0,0.090,1,VC,...,Vacante,2023-01-01,0.00,4,0.0,0,4,2.151354e+08,1.774642e+08,0.82
3115,9990011_2,2024-04-01,VentaFacturadaPortafolioClusterSub,0.82,7.517909e+08,9.120785e+08,9990011.0,0.090,2,VC,...,Vacante,2023-01-01,0.55,4,0.0,0,4,2.134652e+08,2.782575e+08,1.30
3116,9990011_3,2024-04-01,VentaFacturadaPortafolioClusterSub,0.43,9.897435e+07,2.319853e+08,9990011.0,0.060,3,VC,...,Vacante,2023-01-01,0.00,4,0.0,0,4,6.909818e+07,3.372984e+07,0.49
3117,9990011_4,2024-04-01,VentaFacturadaPortafolioClusterSub,1.07,5.362554e+07,5.019802e+07,9990011.0,0.045,4,VC,...,Vacante,2023-01-01,1.70,4,0.0,0,4,1.287129e+07,1.321235e+07,1.03


In [75]:
df_resultado_detallado_previo[
    (df_resultado_detallado_previo['Contexto']=='1111117') &
    (df_resultado_detallado_previo['Variable']=='ObjetivosCualitativos')
]

,Contexto,Fecha,Variable,PorcentajeCumplimiento,Real,Presupuesto,CodigoEmpleado,Porcentaje,Consecutivo,TipoEmpleado,...,Nombre,FechaIngreso,FactorIncentivo,FactorMes,Liquidado,DifMonths,NuevoFactor,ResultadoPresupuesto,ResultadoRealMes,PorcentajeCumplimientoMes
412,1111117,2024-01-01,ObjetivosCualitativos,0.94,0.0,0.0,1111117.0,0.15,0,VC,...,Julieth,2023-01-01,0.85,1,318750.0,0,1,0.0,0.0,0.0
1027,1111117,2024-02-01,ObjetivosCualitativos,0.94,0.0,0.0,1111117.0,0.15,0,VC,...,JULIETH,2023-01-01,0.85,2,637500.0,0,2,0.0,0.0,0.0
1634,1111117,2024-03-01,ObjetivosCualitativos,0.80,0.0,0.0,1111117.0,0.15,0,VC,...,Julieth,2023-01-01,0.50,3,562500.0,0,3,0.0,0.0,0.0
2470,1111117,2024-04-01,ObjetivosCualitativos,0.80,0.0,0.0,1111117.0,0.15,0,VC,...,Julieth,2023-01-01,0.50,4,750000.0,0,4,0.0,0.0,0.0


In [76]:
# Se une el resultadio histórico con el actual, de tal forma que al histórico se le agregan únicamenet 
# los resultados correspondientes al mes que se está liquidando
resultado_detallado_historico = df_resultado_detallado_previo.copy()
# Filtrar el DataFrame para encontrar las filas donde 'CodigoEmpleado' termina con '.0'
resultado_detallado_historico[resultado_detallado_historico['CodigoEmpleado'].astype(str).str.endswith('.0')].head()
resultado_detallado_historico['CodigoEmpleado'] = resultado_detallado_historico['CodigoEmpleado'].astype(str).str.replace(r'(\d+)\.0$', r'\1', regex=True)
resultado_detallado_historico


,Contexto,Fecha,Variable,PorcentajeCumplimiento,Real,Presupuesto,CodigoEmpleado,Porcentaje,Consecutivo,TipoEmpleado,...,Nombre,FechaIngreso,FactorIncentivo,FactorMes,Liquidado,DifMonths,NuevoFactor,ResultadoPresupuesto,ResultadoRealMes,PorcentajeCumplimientoMes
0,Pais,2024-02-01,VentaCobradaBraunPais,0.88,3.221882e+10,3.663783e+10,1010166177,0.300,0,VE,...,William Andres,2023-12-01,0.70,2,1050000.0,0,2,1.913977e+10,1.551444e+10,0.81
1,1010166177_1,2024-02-01,VentaFacturadaPortafolioZonaClusterSub,1.66,3.157478e+08,1.900081e+08,1010166177,0.150,1,VE,...,William Andres,2023-12-01,2.50,2,1875000.0,0,2,9.500404e+07,2.604769e+08,2.74
2,1010166177,2024-02-01,VentaCobradaPortafolioZonaCluster,2.89,3.886020e+08,1.345891e+08,1010166177,0.250,0,VE,...,William Andres,2023-12-01,2.50,2,3125000.0,0,2,6.729453e+07,2.759412e+08,4.10
3,Pais,2024-03-01,VentaCobradaBraunPais,0.86,4.803727e+10,5.604000e+10,1010166177,0.300,0,VE,...,William Andres,2023-12-01,0.65,3,1462500.0,0,3,1.940218e+10,1.581845e+10,0.82
4,Pais,2024-02-01,VentaCobradaBraunPais,0.88,3.221882e+10,3.663783e+10,1014204375,0.300,0,VE,...,Caterin Julieth,2024-01-01,0.70,2,1050000.0,0,2,1.913977e+10,1.551444e+10,0.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3114,9990011_1,2024-04-01,VentaFacturadaPortafolioClusterSub,0.79,7.307934e+08,9.299402e+08,9990011,0.090,1,VC,...,Vacante,2023-01-01,0.00,4,0.0,0,4,2.151354e+08,1.774642e+08,0.82
3115,9990011_2,2024-04-01,VentaFacturadaPortafolioClusterSub,0.82,7.517909e+08,9.120785e+08,9990011,0.090,2,VC,...,Vacante,2023-01-01,0.55,4,0.0,0,4,2.134652e+08,2.782575e+08,1.30
3116,9990011_3,2024-04-01,VentaFacturadaPortafolioClusterSub,0.43,9.897435e+07,2.319853e+08,9990011,0.060,3,VC,...,Vacante,2023-01-01,0.00,4,0.0,0,4,6.909818e+07,3.372984e+07,0.49
3117,9990011_4,2024-04-01,VentaFacturadaPortafolioClusterSub,1.07,5.362554e+07,5.019802e+07,9990011,0.045,4,VC,...,Vacante,2023-01-01,1.70,4,0.0,0,4,1.287129e+07,1.321235e+07,1.03


In [77]:
# resultado_detallado_historico.info()
# resultado_detallado_historico.to_excel('resultado_detallado_historico.xlsx')

In [78]:
fecha_liquidacion

'2024-05-01'

In [79]:
last_date_historico = resultado_detallado_historico['Fecha'].max()
last_date_historico = None if pd.isnull(last_date_historico) else last_date_historico
last_date_historico

Timestamp('2024-04-01 00:00:00')

In [80]:
# maestra_resultados_por_empleado[
#     maestra_resultados_por_empleado['Fecha']>last_date_historico
# ]

In [81]:
resultado_detallado_historico[
    (resultado_detallado_historico['Contexto']=='1111117') &
    (resultado_detallado_historico['Variable']=='ObjetivosCualitativos')
]

,Contexto,Fecha,Variable,PorcentajeCumplimiento,Real,Presupuesto,CodigoEmpleado,Porcentaje,Consecutivo,TipoEmpleado,...,Nombre,FechaIngreso,FactorIncentivo,FactorMes,Liquidado,DifMonths,NuevoFactor,ResultadoPresupuesto,ResultadoRealMes,PorcentajeCumplimientoMes
412,1111117,2024-01-01,ObjetivosCualitativos,0.94,0.0,0.0,1111117,0.15,0,VC,...,Julieth,2023-01-01,0.85,1,318750.0,0,1,0.0,0.0,0.0
1027,1111117,2024-02-01,ObjetivosCualitativos,0.94,0.0,0.0,1111117,0.15,0,VC,...,JULIETH,2023-01-01,0.85,2,637500.0,0,2,0.0,0.0,0.0
1634,1111117,2024-03-01,ObjetivosCualitativos,0.80,0.0,0.0,1111117,0.15,0,VC,...,Julieth,2023-01-01,0.50,3,562500.0,0,3,0.0,0.0,0.0
2470,1111117,2024-04-01,ObjetivosCualitativos,0.80,0.0,0.0,1111117,0.15,0,VC,...,Julieth,2023-01-01,0.50,4,750000.0,0,4,0.0,0.0,0.0


In [82]:
maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['Contexto']=='1111117') &
    (maestra_resultados_por_empleado['Variable']=='ObjetivosCualitativos')
]

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,SalarioVariable,FactorIncentivo,FactorMes,Liquidado,DifMonths,NuevoFactor,NuevoFactorBK,ResultadoPresupuesto,ResultadoRealMes,PorcentajeCumplimientoMes
510,510,1111117,2024-01-01,0.8,ObjetivosCualitativos,0.0,0.0,0.0,0.0,0.0,...,2500000.0,0.5,1,187500.0,0,1,1,0.0,0.0,0.0
511,511,1111117,2024-02-01,0.8,ObjetivosCualitativos,0.0,0.0,0.0,0.0,0.0,...,2500000.0,0.5,2,375000.0,0,2,2,0.0,0.0,0.0
512,512,1111117,2024-03-01,0.8,ObjetivosCualitativos,0.0,0.0,0.0,0.0,0.0,...,2500000.0,0.5,3,562500.0,0,3,3,0.0,0.0,0.0
513,513,1111117,2024-04-01,0.8,ObjetivosCualitativos,0.0,0.0,0.0,0.0,0.0,...,2500000.0,0.5,4,750000.0,0,4,4,0.0,0.0,0.0
514,514,1111117,2024-05-01,0.8,ObjetivosCualitativos,0.0,0.0,0.0,0.0,0.0,...,2500000.0,0.5,5,937500.0,0,5,5,0.0,0.0,0.0


In [83]:
maestra_resultados_por_empleado = maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['Fecha']<=fecha_liquidacion)
]

maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['Contexto']=='1111117') &
    (maestra_resultados_por_empleado['Variable']=='ObjetivosCualitativos')
]

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,SalarioVariable,FactorIncentivo,FactorMes,Liquidado,DifMonths,NuevoFactor,NuevoFactorBK,ResultadoPresupuesto,ResultadoRealMes,PorcentajeCumplimientoMes
510,510,1111117,2024-01-01,0.8,ObjetivosCualitativos,0.0,0.0,0.0,0.0,0.0,...,2500000.0,0.5,1,187500.0,0,1,1,0.0,0.0,0.0
511,511,1111117,2024-02-01,0.8,ObjetivosCualitativos,0.0,0.0,0.0,0.0,0.0,...,2500000.0,0.5,2,375000.0,0,2,2,0.0,0.0,0.0
512,512,1111117,2024-03-01,0.8,ObjetivosCualitativos,0.0,0.0,0.0,0.0,0.0,...,2500000.0,0.5,3,562500.0,0,3,3,0.0,0.0,0.0
513,513,1111117,2024-04-01,0.8,ObjetivosCualitativos,0.0,0.0,0.0,0.0,0.0,...,2500000.0,0.5,4,750000.0,0,4,4,0.0,0.0,0.0
514,514,1111117,2024-05-01,0.8,ObjetivosCualitativos,0.0,0.0,0.0,0.0,0.0,...,2500000.0,0.5,5,937500.0,0,5,5,0.0,0.0,0.0


In [84]:
if last_date_historico:
    maestra_resultados_por_empleado = maestra_resultados_por_empleado[
        maestra_resultados_por_empleado['Fecha']>last_date_historico
    ]

In [85]:
# last_date_historico = resultado_detallado_historico['Fecha'].max()
# last_date_historico = None if pd.isnull(last_date_historico) else last_date_historico

try:
    maestra_resultados_por_empleado = maestra_resultados_por_empleado[
        (maestra_resultados_por_empleado['Fecha']<=fecha_liquidacion)
    ]
    if last_date_historico:
        maestra_resultados_por_empleado = maestra_resultados_por_empleado[
            maestra_resultados_por_empleado['Fecha']>last_date_historico
        ]
except:
    raise

In [86]:
maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['Contexto']=='1111117') &
    (maestra_resultados_por_empleado['Variable']=='ObjetivosCualitativos')
]

,Unnamed: 0,Contexto,Fecha,PorcentajeCumplimiento,Variable,Real,Presupuesto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,...,SalarioVariable,FactorIncentivo,FactorMes,Liquidado,DifMonths,NuevoFactor,NuevoFactorBK,ResultadoPresupuesto,ResultadoRealMes,PorcentajeCumplimientoMes
514,514,1111117,2024-05-01,0.8,ObjetivosCualitativos,0.0,0.0,0.0,0.0,0.0,...,2500000.0,0.5,5,937500.0,0,5,5,0.0,0.0,0.0


In [87]:
maestra_resultados_por_empleado = pd.concat([resultado_detallado_historico, maestra_resultados_por_empleado])
maestra_resultados_por_empleado = maestra_resultados_por_empleado.sort_values(by=['CodigoEmpleado', 'Fecha'])
maestra_resultados_por_empleado

,Contexto,Fecha,Variable,PorcentajeCumplimiento,Real,Presupuesto,CodigoEmpleado,Porcentaje,Consecutivo,TipoEmpleado,...,ResultadoRealMes,PorcentajeCumplimientoMes,Unnamed: 0,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,Nombre variable,Nombre consecutivo,_merge,NuevoFactorBK
269,1000972927,2024-01-01,ObjetivosCualitativos,0.94,0.000000e+00,0.000000e+00,1000972927,0.20,0.0,MK,...,0.000000e+00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270,mk_12_1,2024-01-01,VentaFacturadaUnidadNegocioSub,1.07,9.692130e+09,9.072987e+09,1000972927,0.24,1.0,MK,...,9.692130e+09,1.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
271,mk_12_2,2024-01-01,VentaFacturadaUnidadNegocioSub,0.86,2.906090e+09,3.364754e+09,1000972927,0.14,2.0,MK,...,2.906090e+09,0.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
272,mk_12_3,2024-01-01,VentaFacturadaUnidadNegocioSub,0.58,5.527091e+07,9.500404e+07,1000972927,0.02,3.0,MK,...,5.527091e+07,0.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
898,1000972927,2024-02-01,ObjetivosCualitativos,0.94,0.000000e+00,0.000000e+00,1000972927,0.20,0.0,MK,...,0.000000e+00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8948,5041308,2024-05-01,VentaCobradaPortafolioCluster,0.95,4.959825e+09,5.214068e+09,nan,NaN,NaN,NaN,...,1.765378e+09,1.79,147.0,0.0,0.0,0.0,NaN,NaN,left_only,5.0
8953,5041384,2024-05-01,VentaCobradaPortafolioCluster,0.96,2.597681e+09,2.704102e+09,nan,NaN,NaN,NaN,...,4.728069e+08,0.92,152.0,0.0,0.0,0.0,NaN,NaN,left_only,5.0
8958,5041387,2024-05-01,VentaCobradaPortafolioCluster,0.87,2.757288e+09,3.180014e+09,nan,NaN,NaN,NaN,...,7.310749e+08,1.22,157.0,0.0,0.0,0.0,NaN,NaN,left_only,5.0
8963,5043868,2024-05-01,VentaCobradaPortafolioCluster,1.02,4.090969e+09,4.014532e+09,nan,NaN,NaN,NaN,...,1.510337e+09,1.99,162.0,0.0,0.0,0.0,NaN,NaN,left_only,5.0


In [88]:
maestra_resultados_por_empleado['Variable'].drop_duplicates()

269                      ObjetivosCualitativos
270             VentaFacturadaUnidadNegocioSub
1448                              EBITCOGSPais
274                      VentaCobradaBraunPais
275     VentaFacturadaPortafolioZonaClusterSub
276          VentaCobradaPortafolioZonaCluster
903                 VentaCobradaPortafolioPais
284                    VentaFacturadaBraunPais
1462       RentabilidadUnidadNegocioCM2COGSSUB
1466                              KpisSurgical
1493                VentaFacturadaBraunPaisSUB
1494                           EBITCOGSPaisSUB
27               VentaCobradaPortafolioCluster
378                   VentaCobradaBraunCluster
380         VentaFacturadaPortafolioClusterSub
417           VentaFacturadaPortafolioRenalSub
1640               VentaCobradaPortafolioRenal
238                  VentaFacturadaDivisionSub
1785            RentabilidadDivisionCM2COGsSub
247         VentaFacturadaGrupoClientesKAMSSub
538                  VentaFacturadaZonaPaisSub
704          

In [89]:
maestra_resultados_por_empleado[
     (maestra_resultados_por_empleado['Fecha']=='2024-05-01') &
    (maestra_resultados_por_empleado['Contexto']=='1111117') &
    (maestra_resultados_por_empleado['Variable']=='ObjetivosCualitativos')
]

,Contexto,Fecha,Variable,PorcentajeCumplimiento,Real,Presupuesto,CodigoEmpleado,Porcentaje,Consecutivo,TipoEmpleado,...,ResultadoRealMes,PorcentajeCumplimientoMes,Unnamed: 0,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,Nombre variable,Nombre consecutivo,_merge,NuevoFactorBK
514,1111117,2024-05-01,ObjetivosCualitativos,0.8,0.0,0.0,1111117,0.15,0.0,VC,...,0.0,0.0,514.0,0.0,0.0,0.0,Objetivos cualitativos,NaN,both,5.0


### Maestra Resultados por Empleado Centro Costo vs Resultado Detallado Histórico

In [90]:
maestra_resultados_por_empleado_centro_costo_final['Real'].fillna(0,inplace=True)
maestra_resultados_por_empleado_centro_costo_final['Presupuesto'].fillna(0,inplace=True)
maestra_resultados_por_empleado_centro_costo_final['ResultadoPresupuesto'] = 0
maestra_resultados_por_empleado_centro_costo_final['ResultadoRealMes'] = 0
maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimientoMes'] = 0
maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimientoMes'] = maestra_resultados_por_empleado_centro_costo_final['PorcentajeCumplimientoMes'].astype('float')
maestra_resultados_por_empleado_centro_costo_final = maestra_resultados_por_empleado_centro_costo_final.groupby(['CodigoEmpleado', 'Variable', 'Contexto'], group_keys=False).apply(calcular_resultado_mes)

maestra_resultados_por_empleado_centro_costo_final.head()

C:\Users\jjgg\AppData\Local\Temp\ipykernel_25464\3155329719.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '116255.29795077215' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group.at[index, 'ResultadoRealMes'] = resultado_real_mes
C:\Users\jjgg\AppData\Local\Temp\ipykernel_25464\3155329719.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '155805.14622926473' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group.at[index, 'ResultadoPresupuesto'] = resultado_presupuesto_mes
C:\Users\jjgg\AppData\Local\Temp\ipykernel_25464\3155329719.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '194414.7029081006' has dtype incompatible with int64, please explicitly cast to a compatible dtype fir

,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,PorcentajeCumplimiento,RealCOP,...,SalarioVariable,FactorIncentivo,FactorMes,Liquidado,DifMonths,NuevoFactor,NuevoFactorBK,ResultadoPresupuesto,ResultadoRealMes,PorcentajeCumplimientoMes
0,5002711_1,2024-01-01,610,HC,323015040,2.838878e+08,2.926409e+08,VentaCobradaZonaPaisSub,0.97,0.0,...,NaN,NaN,1,NaN,0,1,1,2.926409e+08,2.838878e+08,0.97
1,5002711_1,2024-01-01,620,HC,323015045,1.906658e+09,2.109654e+09,VentaCobradaZonaPaisSub,0.90,0.0,...,NaN,NaN,1,NaN,0,1,1,1.817013e+09,1.622770e+09,0.89
2,5002711_1,2024-01-01,630,HC,323015050,1.321169e+09,1.181046e+09,VentaCobradaZonaPaisSub,1.12,0.0,...,NaN,2.5,1,NaN,0,1,1,-9.286073e+08,-5.854891e+08,1.37
3,5002711_1,2024-01-01,640,HC,323015055,0.000000e+00,2.040000e+06,VentaCobradaZonaPaisSub,0.00,0.0,...,NaN,0.0,1,NaN,0,1,1,-1.179006e+09,-1.321169e+09,0.88
4,5002711_1,2024-01-01,680,HC,323015055,2.727908e+08,2.324993e+08,VentaCobradaZonaPaisSub,1.17,0.0,...,NaN,2.5,1,NaN,0,1,1,2.304593e+08,2.727908e+08,1.18


In [91]:
df_resultado_detallado_previo.copy().merge(
    df_area_calculo_sba_centros_costos.rename(columns={'ConsecutivoParrilla':'Consecutivo'}),
    on=['AreaCalculo', 'Consecutivo','TipoEmpleado'],
    how='left'    
)

# Se une el resultadio histórico con el actual, de tal forma que al histórico se le agregan únicamenet 
# los resultados correspondientes al mes que se está liquidando
resultado_detallado_historico = df_resultado_detallado_previo.copy()
last_date_historico = resultado_detallado_historico['Fecha'].max()
last_date_historico = None if pd.isnull(last_date_historico) else last_date_historico

try:
    maestra_resultados_por_empleado_centro_costo_final = maestra_resultados_por_empleado_centro_costo_final[
        (maestra_resultados_por_empleado_centro_costo_final['Fecha']<=fecha_liquidacion)
    ]
    if last_date_historico:
        maestra_resultados_por_empleado_centro_costo_final = maestra_resultados_por_empleado_centro_costo_final[
            maestra_resultados_por_empleado_centro_costo_final['Fecha']>last_date_historico
        ]
except:
    raise

maestra_resultados_por_empleado_centro_costo_final = pd.concat([resultado_detallado_historico, maestra_resultados_por_empleado_centro_costo_final])
maestra_resultados_por_empleado_centro_costo_final = maestra_resultados_por_empleado_centro_costo_final.sort_values(by=['CodigoEmpleado', 'Fecha'])
maestra_resultados_por_empleado_centro_costo_final

,Contexto,Fecha,Variable,PorcentajeCumplimiento,Real,Presupuesto,CodigoEmpleado,Porcentaje,Consecutivo,TipoEmpleado,...,PorcentajeCumplimientoMes,GrupoProducto,Division,CodigoCentroCosto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,Nombre variable,Nombre consecutivo,NuevoFactorBK
2642,mk_12_1,2024-05-01,VentaFacturadaUnidadNegocioSub,1.32,1.286746e+06,9.751253e+05,1000972927,0.24,1.0,MK,...,1.69,610,HC,323015040,5.438393e+09,4.536624e+09,1.198775,Venta Facturada Portafolio a cargo,Medical Care,5.0
2656,mk_12_1,2024-05-01,VentaFacturadaUnidadNegocioSub,1.12,7.548071e+06,6.749920e+06,1000972927,0.24,1.0,MK,...,1.08,620,HC,323015045,3.190170e+10,3.140299e+10,1.015881,Venta Facturada Portafolio a cargo,Medical Care,5.0
2670,mk_12_1,2024-05-01,VentaFacturadaUnidadNegocioSub,1.12,4.401670e+06,3.913573e+06,1000972927,0.24,1.0,MK,...,0.89,630,HC,323015050,1.860353e+10,1.820731e+10,1.021761,Venta Facturada Portafolio a cargo,Medical Care,5.0
2684,mk_12_1,2024-05-01,VentaFacturadaUnidadNegocioSub,0.33,5.150514e+03,1.565703e+04,1000972927,0.24,1.0,MK,...,0.87,640,HC,323015055,2.176849e+07,7.284197e+07,0.298845,Venta Facturada Portafolio a cargo,Medical Care,5.0
2698,mk_12_1,2024-05-01,VentaFacturadaUnidadNegocioSub,1.07,7.942036e+05,7.432677e+05,1000972927,0.24,1.0,MK,...,1.08,680,HC,323015055,3.356678e+09,3.457941e+09,0.970716,Venta Facturada Portafolio a cargo,Medical Care,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11190,79625102_1,2024-05-01,VentaFacturadaPortafolioClusterSub,0.43,5.609398e+08,1.295763e+09,nan,NaN,NaN,NaN,...,0.28,A35,BA,323007330,0.000000e+00,0.000000e+00,0.000000,NaN,NaN,5.0
13567,5004293_1,2024-05-01,VentaFacturadaGrupoClientesKAMSSub,0.91,1.036601e+06,1.136338e+06,nan,NaN,NaN,NaN,...,0.85,630,HC,323015090,4.381161e+09,5.286644e+09,0.828723,NaN,NaN,5.0
13576,5004293_2,2024-05-01,VentaFacturadaGrupoClientesKAMSSub,1.21,3.337287e+06,2.757745e+06,nan,NaN,NaN,NaN,...,1.17,630,HC,323015090,1.410494e+10,1.283000e+10,1.099372,NaN,NaN,5.0
13577,5004293_2,2024-05-01,VentaFacturadaGrupoClientesKAMSSub,1.21,3.337287e+06,2.757745e+06,nan,NaN,NaN,NaN,...,0.00,CDK,HC,323015090,1.410494e+10,1.283000e+10,1.099372,NaN,NaN,5.0


In [92]:
maestra_resultados_por_empleado_centro_costo_final[maestra_resultados_por_empleado_centro_costo_final['CodigoEmpleado'] == '1098612757']


,Contexto,Fecha,Variable,PorcentajeCumplimiento,Real,Presupuesto,CodigoEmpleado,Porcentaje,Consecutivo,TipoEmpleado,...,PorcentajeCumplimientoMes,GrupoProducto,Division,CodigoCentroCosto,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,Nombre variable,Nombre consecutivo,NuevoFactorBK
11214,1098612757_1,2024-05-01,VentaFacturadaBraunPaisSUB,1.07,4.632354e+07,4.341862e+07,1098612757,0.4,1.0,AD1,...,1.06,BBMCO,BBMCO,323004021,0.0,0.0,0.0,VentaFacturadaBraunPaisSUB,BBMCO,5.0


In [93]:
maestra_resultados_por_empleado[maestra_resultados_por_empleado['Variable'] == 'VentaFacturadaGrupoClientesKAMSSub']

,Contexto,Fecha,Variable,PorcentajeCumplimiento,Real,Presupuesto,CodigoEmpleado,Porcentaje,Consecutivo,TipoEmpleado,...,ResultadoRealMes,PorcentajeCumplimientoMes,Unnamed: 0,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,Nombre variable,Nombre consecutivo,_merge,NuevoFactorBK
247,kam_3_1,2024-01-01,VentaFacturadaGrupoClientesKAMSSub,1.00,2.336746e+09,2.346066e+09,42897142,0.16,1.0,KAM,...,2.336746e+09,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248,kam_3_2,2024-01-01,VentaFacturadaGrupoClientesKAMSSub,0.56,4.942717e+08,8.813119e+08,42897142,0.16,2.0,KAM,...,4.942717e+08,0.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249,kam_3_3,2024-01-01,VentaFacturadaGrupoClientesKAMSSub,0.47,1.042911e+08,2.205255e+08,42897142,0.08,3.0,KAM,...,1.042911e+08,0.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1800,kam_3_1,2024-03-01,VentaFacturadaGrupoClientesKAMSSub,0.97,1.788768e+06,1.845593e+06,42897142,0.16,1.0,KAM,...,5.545224e+05,0.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1801,kam_3_2,2024-03-01,VentaFacturadaGrupoClientesKAMSSub,0.91,6.456084e+05,7.058902e+05,42897142,0.16,2.0,KAM,...,2.537998e+05,0.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1802,kam_3_3,2024-03-01,VentaFacturadaGrupoClientesKAMSSub,0.62,1.241859e+05,2.011400e+05,42897142,0.08,3.0,KAM,...,3.903037e+04,0.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2628,kam_3_1,2024-04-01,VentaFacturadaGrupoClientesKAMSSub,1.15,2.800257e+06,2.430285e+06,42897142,0.16,1.0,KAM,...,9.998713e+05,1.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2629,kam_3_2,2024-04-01,VentaFacturadaGrupoClientesKAMSSub,0.83,8.201040e+05,9.886802e+05,42897142,0.16,2.0,KAM,...,1.703025e+05,0.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2630,kam_3_3,2024-04-01,VentaFacturadaGrupoClientesKAMSSub,0.63,1.785315e+05,2.853358e+05,42897142,0.08,3.0,KAM,...,5.353912e+04,0.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8781,kam_3_1,2024-05-01,VentaFacturadaGrupoClientesKAMSSub,1.11,3.522726e+06,3.179182e+06,42897142,0.16,1.0,KAM,...,7.224695e+05,0.96,19.0,1.488870e+10,1.479067e+10,1.01,VentaFacturadaGrupoClientesKAMSSub,Medical Care,both,5.0


In [94]:
maestra_resultados_por_empleado.to_excel('resultados_detallados.xlsx')

In [95]:
maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['CodigoEmpleado'] == '1111117') &
     (maestra_resultados_por_empleado['Variable'] == 'ObjetivosCualitativos')
]

,Contexto,Fecha,Variable,PorcentajeCumplimiento,Real,Presupuesto,CodigoEmpleado,Porcentaje,Consecutivo,TipoEmpleado,...,ResultadoRealMes,PorcentajeCumplimientoMes,Unnamed: 0,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,Nombre variable,Nombre consecutivo,_merge,NuevoFactorBK
412,1111117,2024-01-01,ObjetivosCualitativos,0.94,0.0,0.0,1111117,0.15,0.0,VC,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1027,1111117,2024-02-01,ObjetivosCualitativos,0.94,0.0,0.0,1111117,0.15,0.0,VC,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1634,1111117,2024-03-01,ObjetivosCualitativos,0.80,0.0,0.0,1111117,0.15,0.0,VC,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2470,1111117,2024-04-01,ObjetivosCualitativos,0.80,0.0,0.0,1111117,0.15,0.0,VC,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
514,1111117,2024-05-01,ObjetivosCualitativos,0.80,0.0,0.0,1111117,0.15,0.0,VC,...,0.0,0.0,514.0,0.0,0.0,0.0,Objetivos cualitativos,NaN,both,5.0


In [96]:
maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['TipoEmpleado'] == 'VC') |
    (maestra_resultados_por_empleado['TipoEmpleado'] == 'VE')
]['Variable'].drop_duplicates()

273                      ObjetivosCualitativos
274                      VentaCobradaBraunPais
275     VentaFacturadaPortafolioZonaClusterSub
276          VentaCobradaPortafolioZonaCluster
903                 VentaCobradaPortafolioPais
27               VentaCobradaPortafolioCluster
378                   VentaCobradaBraunCluster
380         VentaFacturadaPortafolioClusterSub
417           VentaFacturadaPortafolioRenalSub
1640               VentaCobradaPortafolioRenal
Name: Variable, dtype: object

In [97]:
maestra_resultados_por_empleado[maestra_resultados_por_empleado['Variable'] == 'RentabilidadUnidadNegocioCM2COGS']

,Contexto,Fecha,Variable,PorcentajeCumplimiento,Real,Presupuesto,CodigoEmpleado,Porcentaje,Consecutivo,TipoEmpleado,...,ResultadoRealMes,PorcentajeCumplimientoMes,Unnamed: 0,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,Nombre variable,Nombre consecutivo,_merge,NuevoFactorBK


In [98]:
maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['Variable'] == 'VentaFacturadaPortafolioRenalSub')
]

,Contexto,Fecha,Variable,PorcentajeCumplimiento,Real,Presupuesto,CodigoEmpleado,Porcentaje,Consecutivo,TipoEmpleado,...,ResultadoRealMes,PorcentajeCumplimientoMes,Unnamed: 0,RealCOP,PresupuestoCOP,PorcentajeCumplimientoCOP,Nombre variable,Nombre consecutivo,_merge,NuevoFactorBK
417,1111117_1,2024-01-01,VentaFacturadaPortafolioRenalSub,0.58,5.527091e+07,9.500404e+07,1111117,0.15,1.0,VC,...,5.527091e+07,0.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1639,1111117_1,2024-03-01,VentaFacturadaPortafolioRenalSub,5.42,1.629182e+09,3.008461e+08,1111117,0.15,1.0,VC,...,1.313434e+09,11.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2475,1111117_1,2024-04-01,VentaFacturadaPortafolioRenalSub,4.24,1.746318e+09,4.116842e+08,1111117,0.15,1.0,VC,...,1.171361e+08,1.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8276,1111117_1,2024-05-01,VentaFacturadaPortafolioRenalSub,3.37,1.814526e+09,5.383562e+08,1111117,0.15,1.0,VC,...,6.820843e+07,0.54,4.0,0.0,0.0,0.0,VentaFacturadaPortafolioRenalSub,Renal Total Nacional (hospitalio + ambulatorio...,both,5.0
718,5032456_1,2024-01-01,VentaFacturadaPortafolioRenalSub,0.58,5.527091e+07,9.500404e+07,5032456,0.15,1.0,VC,...,5.527091e+07,0.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2034,5032456_1,2024-03-01,VentaFacturadaPortafolioRenalSub,5.42,1.629182e+09,3.008461e+08,5032456,0.15,1.0,VC,...,1.313434e+09,11.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2863,5032456_1,2024-04-01,VentaFacturadaPortafolioRenalSub,4.24,1.746318e+09,4.116842e+08,5032456,0.15,1.0,VC,...,1.171361e+08,1.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8281,5032456_1,2024-05-01,VentaFacturadaPortafolioRenalSub,3.37,1.814526e+09,5.383562e+08,5032456,0.15,1.0,VC,...,6.820843e+07,0.54,9.0,0.0,0.0,0.0,VentaFacturadaPortafolioRenalSub,Renal Total Nacional (hospitalio + ambulatorio...,both,5.0


## Se obtienen las liquidaciones

### Creación de resultado_liquidaciones por maestra_resultados_por_empleado

In [99]:
excepciones_pagos_acumulados = {}

excepciones_diferencias = {}

In [132]:
maestra_resultados_por_empleado[
    (maestra_resultados_por_empleado['CodigoEmpleado'] == '291169471') &
    (maestra_resultados_por_empleado['Contexto'] == '291169471') 
#     (maestra_resultados_por_empleado['Fecha'] == '2024-04-01')
][['Contexto', 'CodigoEmpleado', 'SalarioVariable', 'Fecha', 'NuevoFactorBK', 'Liquidado']]

,Contexto,CodigoEmpleado,SalarioVariable,Fecha,NuevoFactorBK,Liquidado
524,291169471,291169471,6000000.0,2024-01-01,NaN,1020000.0
101,291169471,291169471,6000000.0,2024-02-01,NaN,1020000.0
1781,291169471,291169471,6000000.0,2024-03-01,NaN,600000.0
2609,291169471,291169471,6000000.0,2024-04-01,NaN,600000.0
249,291169471,291169471,6000000.0,2024-05-01,5.0,3000000.0


In [133]:
df_incentivos_por_empleado[df_incentivos_por_empleado['CodigoEmpleado'] == '291169471']

,CodigoEmpleado,Fecha,Valor
270,291169471,2024-01-01,0.0
271,291169471,2024-02-01,0.0
272,291169471,2024-03-01,6000000.0
273,291169471,2024-04-01,6000000.0
274,291169471,2024-05-01,0.0


In [135]:
resultado_liquidaciones = maestra_resultados_por_empleado.groupby(
    ['CodigoEmpleado', 'SalarioVariable', 'Fecha', 'NuevoFactorBK'],
    as_index=False
).Liquidado.agg({'Liquidado': 'sum'})
resultado_liquidaciones.rename(columns={'NuevoFactorBK': 'NuevoFactor'}, inplace=True)
resultado_liquidaciones.head()
resultado_liquidaciones = resultado_liquidaciones.merge(
    df_incentivos_por_empleado.rename(columns={'Valor': 'PagoReal'}),
    on = ['CodigoEmpleado', 'Fecha'],
    how = 'right'
)
resultado_liquidaciones.head()

C:\Users\jjgg\AppData\Local\Temp\ipykernel_25464\2546881310.py:4: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  ).Liquidado.agg({'Liquidado': 'sum'})


,CodigoEmpleado,SalarioVariable,Fecha,NuevoFactor,Liquidado,PagoReal
0,1000972927,NaN,2024-01-01,NaN,NaN,150000.0
1,1000972927,NaN,2024-02-01,NaN,NaN,150000.0
2,1000972927,NaN,2024-03-01,NaN,NaN,1218750.0
3,1000972927,NaN,2024-04-01,NaN,NaN,1259860.0
4,1000972927,388000.0,2024-05-01,5.0,3498790.0,0.0


In [134]:
resultado_liquidaciones[resultado_liquidaciones['CodigoEmpleado'] == '291169471']

,CodigoEmpleado,SalarioVariable,Fecha,FactorMes,PagoGarantizado,Liquidado,PagoReal,PagadoFecha
29,291169471,6000000.0,2024-01-01,0.0,SI,6000000.0,6000000.0,0.0
308,291169471,6000000.0,2024-03-01,0.0,SI,6000000.0,6000000.0,0.0
449,291169471,6000000.0,2024-04-01,0.0,SI,6000000.0,6000000.0,0.0
261,291169471,6000000.0,2024-05-01,5.0,NO,50850000.0,50850000.0,0.0


In [104]:
resultado_liquidaciones[
    (resultado_liquidaciones['CodigoEmpleado'] == '16659046')
]

,CodigoEmpleado,SalarioVariable,Fecha,NuevoFactor,Liquidado,PagoReal
260,16659046,NaN,2024-01-01,NaN,NaN,856000.0
261,16659046,NaN,2024-02-01,NaN,NaN,856000.0
262,16659046,NaN,2024-03-01,NaN,NaN,8560000.0
263,16659046,NaN,2024-04-01,NaN,NaN,8838500.0
264,16659046,2286800.0,2024-05-01,5.0,24011400.0,0.0


In [105]:
resultado_liquidaciones = pd.merge(
    left=resultado_liquidaciones,
    right=df_empleados[['CodigoEmpleado', 'FechaIngreso']].drop_duplicates(['CodigoEmpleado']),
    how='left',
    on=['CodigoEmpleado']
)
resultado_liquidaciones = resultado_liquidaciones[resultado_liquidaciones['Fecha'] >= resultado_liquidaciones['FechaIngreso']]
resultado_liquidaciones.drop(columns=['FechaIngreso'], inplace=True)
resultado_liquidaciones.head()

# Cálculo de garantizado (100%) para VEs y VCs, Anticipos (85%) para el resto de empleados
resultado_liquidaciones = resultado_liquidaciones.merge(
    df_empleados[['CodigoEmpleado', 'TipoEmpleado']],
    on='CodigoEmpleado',
    how='left'
)

def calcular_liquidacion_mes(g):
    pago_acumulado = 0
    for i, row in g.iterrows():
        print("mes actual:", row['Fecha'].month)
        if row['Fecha'].month in meses_incentivos:
            print("Pago Real", g.at[i, 'PagoReal'])
            if g.at[i, 'PagoReal'] == 0:
                print("Pago Real es 0", g.at[i, 'PagoReal'])
                if i == 0:
                    print("i es igual a 0")
                    g.at[i, 'PagoReal'] = g.at[i, 'Liquidado']
                else:
                    print("i no es igual a 0")
                    g.at[i, 'PagoReal'] = g.at[i, 'Liquidado'] - pago_acumulado
                print("Pago Real actualizado", g.at[i, 'PagoReal'])
            if g.at[i, 'PagoReal'] < 0:
                g.at[i, 'PagoReal'] = 0.0
        print("pago_acumuladol_antes:", pago_acumulado)
        pago_acumulado += g.at[i, 'PagoReal']
        print("pago_acumuladol_después:", pago_acumulado)
        
        excepcion = excepciones_pagos_acumulados.get(row['CodigoEmpleado'], None)
        if excepcion and row['Fecha'] == excepcion['Fecha']:
            pago_acumulado = excepcion['ValorPagoAcumulado']
        
        excepcion_diferencia = excepciones_diferencias.get(row['CodigoEmpleado'], None)
        if excepcion_diferencia and row['Fecha'] == excepcion_diferencia['Fecha']:
            pago_acumulado = pago_acumulado + excepcion_diferencia['ValorDiferencia']
        
    return g
resultado_liquidaciones = resultado_liquidaciones.sort_values(by=['CodigoEmpleado', 'Fecha'])
resultado_liquidaciones.reset_index(inplace=True, drop=True)
resultado_liquidaciones['PagoReal'].fillna(0.0, inplace=True)
resultado_liquidaciones['Liquidado'].fillna(0.0, inplace=True)
resultado_liquidaciones

,CodigoEmpleado,SalarioVariable,Fecha,NuevoFactor,Liquidado,PagoReal,TipoEmpleado
0,1000972927,NaN,2024-01-01,NaN,0.00,150000.000,MK
1,1000972927,NaN,2024-02-01,NaN,0.00,150000.000,MK
2,1000972927,NaN,2024-03-01,NaN,0.00,1218750.000,MK
3,1000972927,NaN,2024-04-01,NaN,0.00,1259860.000,MK
4,1000972927,388000.0,2024-05-01,5.0,3498790.00,0.000,MK
...,...,...,...,...,...,...,...
651,94487610,NaN,2024-01-01,NaN,0.00,1000000.000,VE
652,94487610,NaN,2024-02-01,NaN,0.00,1000000.000,VE
653,94487610,NaN,2024-03-01,NaN,0.00,2878000.000,VE
654,94487610,NaN,2024-04-01,NaN,0.00,5369695.625,VE


In [106]:
resultado_liquidaciones[resultado_liquidaciones['CodigoEmpleado'] == '16659046']

,CodigoEmpleado,SalarioVariable,Fecha,NuevoFactor,Liquidado,PagoReal,TipoEmpleado
251,16659046,NaN,2024-01-01,NaN,0.0,856000.0,AD
252,16659046,NaN,2024-02-01,NaN,0.0,856000.0,AD
253,16659046,NaN,2024-03-01,NaN,0.0,8560000.0,AD
254,16659046,NaN,2024-04-01,NaN,0.0,8838500.0,AD
255,16659046,2286800.0,2024-05-01,5.0,24011400.0,0.0,AD


In [107]:
resultado_liquidaciones[
    (resultado_liquidaciones['CodigoEmpleado'] == '16659046')
]

,CodigoEmpleado,SalarioVariable,Fecha,NuevoFactor,Liquidado,PagoReal,TipoEmpleado
251,16659046,NaN,2024-01-01,NaN,0.0,856000.0,AD
252,16659046,NaN,2024-02-01,NaN,0.0,856000.0,AD
253,16659046,NaN,2024-03-01,NaN,0.0,8560000.0,AD
254,16659046,NaN,2024-04-01,NaN,0.0,8838500.0,AD
255,16659046,2286800.0,2024-05-01,5.0,24011400.0,0.0,AD


In [108]:
# resultado_liquidaciones_copy = resultado_liquidaciones[
#     (resultado_liquidaciones['CodigoEmpleado'] == '16659046')
# ].groupby(['CodigoEmpleado'], group_keys=False).apply(calcular_liquidacion_mes)
# resultado_liquidaciones_copy[resultado_liquidaciones_copy['CodigoEmpleado'] == '16659046']

In [109]:
#resultado_liquidaciones = resultado_liquidaciones.groupby(['CodigoEmpleado'], group_keys=False).apply(calcular_liquidacion_mes)
resultado_liquidaciones.rename(columns={'NuevoFactor': 'FactorMes'}, inplace=True)
resultado_liquidaciones['PagoGarantizado'] = 'NO'

resultado_liquidaciones['PagoReal'] = np.where(
    (
        (resultado_liquidaciones['Fecha'] >= '2022-01-01') &
        (resultado_liquidaciones['Fecha'] <= '2022-02-01') &
        (resultado_liquidaciones['TipoEmpleado'].isin(['VE', 'VC']))
    ),
    resultado_liquidaciones['SalarioVariable'],
    resultado_liquidaciones['PagoReal']
)
resultado_liquidaciones['PagoReal'] = np.where(
    (
        (resultado_liquidaciones['Fecha'] >= '2022-01-01') &
        (resultado_liquidaciones['Fecha'] <= '2022-02-01') &
        (resultado_liquidaciones['TipoEmpleado'].isin(['AD', 'AD1', 'AD2', 'AD3', 'AD4', 'AD5', 'GC', 'GD', 'GU', 'KAM', 'MK']))
    ),
    resultado_liquidaciones['SalarioVariable'] * 0.85,
    resultado_liquidaciones['PagoReal']
)
resultado_liquidaciones.drop(columns=['TipoEmpleado'], inplace=True)

resultado_liquidaciones.head(12)

,CodigoEmpleado,SalarioVariable,Fecha,FactorMes,Liquidado,PagoReal,PagoGarantizado
0,1000972927,NaN,2024-01-01,NaN,0.0,150000.0,NO
1,1000972927,NaN,2024-02-01,NaN,0.0,150000.0,NO
2,1000972927,NaN,2024-03-01,NaN,0.0,1218750.0,NO
3,1000972927,NaN,2024-04-01,NaN,0.0,1259860.0,NO
4,1000972927,388000.0,2024-05-01,5.0,3498790.0,0.0,NO
5,1010166177,NaN,2024-01-01,NaN,0.0,1250000.0,NO
6,1010166177,NaN,2024-02-01,NaN,0.0,1250000.0,NO
7,1010166177,NaN,2024-03-01,NaN,0.0,11187500.0,NO
8,1010166177,NaN,2024-04-01,NaN,0.0,1587500.0,NO
9,1010166177,2500000.0,2024-05-01,5.0,16343750.0,0.0,NO


In [110]:
resultado_liquidaciones[resultado_liquidaciones['CodigoEmpleado'] == '16659046']

,CodigoEmpleado,SalarioVariable,Fecha,FactorMes,Liquidado,PagoReal,PagoGarantizado
251,16659046,NaN,2024-01-01,NaN,0.0,856000.0,NO
252,16659046,NaN,2024-02-01,NaN,0.0,856000.0,NO
253,16659046,NaN,2024-03-01,NaN,0.0,8560000.0,NO
254,16659046,NaN,2024-04-01,NaN,0.0,8838500.0,NO
255,16659046,2286800.0,2024-05-01,5.0,24011400.0,0.0,NO


In [111]:
resultado_liquidaciones[(resultado_liquidaciones['CodigoEmpleado'] == '291169471') 
#| (resultado_liquidaciones['CodigoEmpleado'] == '1111135')
#| (resultado_liquidaciones['CodigoEmpleado'] == '16659046')
]

,CodigoEmpleado,SalarioVariable,Fecha,FactorMes,Liquidado,PagoReal,PagoGarantizado
261,291169471,6000000.0,2024-05-01,5.0,50850000.0,0.0,NO


### Creación de resultado_liquidaciones por maestra_resultados_por_empleado_centro_costo

In [112]:
resultado_liquidaciones_centro_costo = maestra_resultados_por_empleado_centro_costo_final.groupby(
    ['CodigoEmpleado', 'CodigoCentroCosto','SalarioVariable', 'Fecha', 'NuevoFactorBK'],
    as_index=False
).Liquidado.agg({'Liquidado': 'sum'})
resultado_liquidaciones_centro_costo.rename(columns={'NuevoFactorBK': 'NuevoFactor'}, inplace=True)
resultado_liquidaciones_centro_costo.head()
resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo.merge(
    df_incentivos_por_empleado.rename(columns={'Valor': 'PagoReal'}),
    on = ['CodigoEmpleado', 'Fecha'],
    how = 'right'
)

resultado_liquidaciones_centro_costo = pd.merge(
    left=resultado_liquidaciones_centro_costo,
    right=df_empleados[['CodigoEmpleado', 'FechaIngreso']].drop_duplicates(['CodigoEmpleado']),
    how='left',
    on=['CodigoEmpleado']
)

resultado_liquidaciones_centro_costo.head()

C:\Users\jjgg\AppData\Local\Temp\ipykernel_25464\1216373276.py:4: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  ).Liquidado.agg({'Liquidado': 'sum'})


,CodigoEmpleado,CodigoCentroCosto,SalarioVariable,Fecha,NuevoFactor,Liquidado,PagoReal,FechaIngreso
0,1000972927,NaN,NaN,2024-01-01,NaN,NaN,150000.0,2023-07-01
1,1000972927,NaN,NaN,2024-02-01,NaN,NaN,150000.0,2023-07-01
2,1000972927,NaN,NaN,2024-03-01,NaN,NaN,1218750.0,2023-07-01
3,1000972927,NaN,NaN,2024-04-01,NaN,NaN,1259860.0,2023-07-01
4,1000972927,323007260,388000.0,2024-05-01,5.0,190120.0,0.0,2023-07-01


In [113]:
resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo[resultado_liquidaciones_centro_costo['Fecha'] >= resultado_liquidaciones_centro_costo['FechaIngreso']]
resultado_liquidaciones_centro_costo.drop(columns=['FechaIngreso'], inplace=True)
resultado_liquidaciones_centro_costo.head()

# Cálculo de garantizado (100%) para VEs y VCs, Anticipos (85%) para el resto de empleados
resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo.merge(
    df_empleados[['CodigoEmpleado', 'TipoEmpleado']],
    on='CodigoEmpleado',
    how='left'
)

def calcular_liquidacion_mes_centro_costo(g):
    pago_acumulado = 0
    for i, row in g.iterrows():
        if row['Fecha'].month in meses_incentivos:
            if g.at[i, 'PagoReal'] == 0:
                if i == 0:
                    g.at[i, 'PagoReal'] = g.at[i, 'Liquidado']
                else:
                    g.at[i, 'PagoReal'] = g.at[i, 'Liquidado'] - pago_acumulado
            
            if g.at[i, 'PagoReal'] < 0:
                g.at[i, 'PagoReal'] = 0.0
        pago_acumulado += g.at[i, 'PagoReal']
        
        excepcion = excepciones_pagos_acumulados.get(row['CodigoEmpleado'], None)
        if excepcion and row['Fecha'] == excepcion['Fecha']:
            pago_acumulado = excepcion['ValorPagoAcumulado']
        
        excepcion_diferencia = excepciones_diferencias.get(row['CodigoEmpleado'], None)
        if excepcion_diferencia and row['Fecha'] == excepcion_diferencia['Fecha']:
            pago_acumulado = pago_acumulado + excepcion_diferencia['ValorDiferencia']
        
    return g
resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo.sort_values(by=['CodigoEmpleado', 'Fecha'])
resultado_liquidaciones_centro_costo.reset_index(inplace=True, drop=True)
resultado_liquidaciones_centro_costo['PagoReal'].fillna(0.0, inplace=True)
resultado_liquidaciones_centro_costo['Liquidado'].fillna(0.0, inplace=True)
resultado_liquidaciones_centro_costo

,CodigoEmpleado,CodigoCentroCosto,SalarioVariable,Fecha,NuevoFactor,Liquidado,PagoReal,TipoEmpleado
0,1000972927,NaN,NaN,2024-01-01,NaN,0.00,150000.000,MK
1,1000972927,NaN,NaN,2024-02-01,NaN,0.00,150000.000,MK
2,1000972927,NaN,NaN,2024-03-01,NaN,0.00,1218750.000,MK
3,1000972927,NaN,NaN,2024-04-01,NaN,0.00,1259860.000,MK
4,1000972927,323007260,388000.0,2024-05-01,5.0,190120.00,0.000,MK
...,...,...,...,...,...,...,...,...
1049,94487610,NaN,NaN,2024-01-01,NaN,0.00,1000000.000,VE
1050,94487610,NaN,NaN,2024-02-01,NaN,0.00,1000000.000,VE
1051,94487610,NaN,NaN,2024-03-01,NaN,0.00,2878000.000,VE
1052,94487610,NaN,NaN,2024-04-01,NaN,0.00,5369695.625,VE


In [114]:
resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo.groupby(['CodigoEmpleado'], group_keys=False).apply(calcular_liquidacion_mes_centro_costo)
resultado_liquidaciones_centro_costo.rename(columns={'NuevoFactor': 'FactorMes'}, inplace=True)
resultado_liquidaciones_centro_costo['PagoGarantizado'] = 'NO'

resultado_liquidaciones_centro_costo['PagoReal'] = np.where(
    (
        (resultado_liquidaciones_centro_costo['Fecha'] >= '2022-01-01') &
        (resultado_liquidaciones_centro_costo['Fecha'] <= '2022-02-01') &
        (resultado_liquidaciones_centro_costo['TipoEmpleado'].isin(['VE', 'VC']))
    ),
    resultado_liquidaciones_centro_costo['SalarioVariable'],
    resultado_liquidaciones_centro_costo['PagoReal']
)
resultado_liquidaciones_centro_costo['PagoReal'] = np.where(
    (
        (resultado_liquidaciones_centro_costo['Fecha'] >= '2022-01-01') &
        (resultado_liquidaciones_centro_costo['Fecha'] <= '2022-02-01') &
        (resultado_liquidaciones_centro_costo['TipoEmpleado'].isin(['AD', 'AD1', 'AD2', 'AD3', 'AD4', 'AD5', 'GC', 'GD', 'GU', 'KAM', 'MK']))
    ),
    resultado_liquidaciones_centro_costo['SalarioVariable'] * 0.85,
    resultado_liquidaciones_centro_costo['PagoReal']
)
resultado_liquidaciones_centro_costo.drop(columns=['TipoEmpleado'], inplace=True)

resultado_liquidaciones_centro_costo.head(12)

,CodigoEmpleado,CodigoCentroCosto,SalarioVariable,Fecha,FactorMes,Liquidado,PagoReal,PagoGarantizado
0,1000972927,NaN,NaN,2024-01-01,NaN,0.0,150000.0,NO
1,1000972927,NaN,NaN,2024-02-01,NaN,0.0,150000.0,NO
2,1000972927,NaN,NaN,2024-03-01,NaN,0.0,1218750.0,NO
3,1000972927,NaN,NaN,2024-04-01,NaN,0.0,1259860.0,NO
4,1000972927,323007260,388000.0,2024-05-01,5.0,190120.0,0.0,NO
5,1000972927,323007270,388000.0,2024-05-01,5.0,516040.0,0.0,NO
6,1000972927,323007290,388000.0,2024-05-01,5.0,848750.0,0.0,NO
7,1000972927,323007300,388000.0,2024-05-01,5.0,0.0,0.0,NO
8,1000972927,323007337,388000.0,2024-05-01,5.0,194000.0,0.0,NO
9,1000972927,323007350,388000.0,2024-05-01,5.0,0.0,0.0,NO


## Se obtienen los garantizados

### Sin centro de costo

In [115]:
empleados_garantizado = df_empleados[df_empleados['FechaIngreso'] > fecha_liquidacion][['CodigoEmpleado', 'SalarioVariable', 'FechaIngreso']]
fechas_resultado_liquidaciones = resultado_liquidaciones[['Fecha']].drop_duplicates()

empleados_garantizado['Key'] = 1
fechas_resultado_liquidaciones['Key'] = 1

liquidaciones_garantizados = pd.merge(
    left=fechas_resultado_liquidaciones.sort_values(['Fecha']),
    right=empleados_garantizado,
    how='outer',
    on=['Key']
).drop(columns=['Key'])

liquidaciones_garantizados = liquidaciones_garantizados[liquidaciones_garantizados['CodigoEmpleado'].notnull()]

liquidaciones_garantizados['DifMonths'] = (liquidaciones_garantizados['FechaIngreso'] - liquidaciones_garantizados['Fecha']) // pd.Timedelta(days=30)
liquidaciones_garantizados['DifMonths'] = liquidaciones_garantizados['DifMonths'].apply(lambda x: round(x, 0))
liquidaciones_garantizados['DifMonths'] = liquidaciones_garantizados['DifMonths'].astype('int')
liquidaciones_garantizados = liquidaciones_garantizados[liquidaciones_garantizados['DifMonths'] <= 4]

liquidaciones_garantizados['FactorMes'] = 0
liquidaciones_garantizados['Liquidado'] = liquidaciones_garantizados['SalarioVariable']
liquidaciones_garantizados['PagoReal'] = liquidaciones_garantizados['SalarioVariable']
liquidaciones_garantizados['PagoGarantizado'] = 'SI'

liquidaciones_garantizados.drop(columns=['DifMonths', 'FechaIngreso'], inplace=True)

liquidaciones_garantizados.head()

,Fecha,CodigoEmpleado,SalarioVariable,FactorMes,Liquidado,PagoReal,PagoGarantizado
4,2024-02-01,1015432392,500000.0,0,500000.0,500000.0,SI
5,2024-02-01,797582501,2000000.0,0,2000000.0,2000000.0,SI
8,2024-03-01,1015432392,500000.0,0,500000.0,500000.0,SI
9,2024-03-01,797582501,2000000.0,0,2000000.0,2000000.0,SI
10,2024-03-01,80037193,600000.0,0,600000.0,600000.0,SI


### Con centro de costo

In [116]:
empleados_garantizado = df_empleados[df_empleados['FechaIngreso'] > fecha_liquidacion][['CodigoEmpleado', 'SalarioVariable', 'FechaIngreso']]
fechas_resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo[['Fecha']].drop_duplicates()

empleados_garantizado['Key'] = 1
fechas_resultado_liquidaciones_centro_costo['Key'] = 1

liquidaciones_garantizados_centro_costo = pd.merge(
    left=fechas_resultado_liquidaciones_centro_costo.sort_values(['Fecha']),
    right=empleados_garantizado,
    how='outer',
    on=['Key']
).drop(columns=['Key'])

liquidaciones_garantizados_centro_costo = liquidaciones_garantizados_centro_costo[liquidaciones_garantizados_centro_costo['CodigoEmpleado'].notnull()]

liquidaciones_garantizados_centro_costo['DifMonths'] = ((liquidaciones_garantizados_centro_costo.FechaIngreso - liquidaciones_garantizados_centro_costo.Fecha)) // pd.Timedelta(days=30)
liquidaciones_garantizados_centro_costo['DifMonths'] = liquidaciones_garantizados_centro_costo['DifMonths'].apply(lambda x: round(x, 0))
liquidaciones_garantizados_centro_costo['DifMonths'] = liquidaciones_garantizados_centro_costo['DifMonths'].astype('int')
liquidaciones_garantizados_centro_costo = liquidaciones_garantizados_centro_costo[liquidaciones_garantizados_centro_costo['DifMonths'] <= 4]

liquidaciones_garantizados_centro_costo['FactorMes'] = 0
liquidaciones_garantizados_centro_costo['Liquidado'] = liquidaciones_garantizados_centro_costo['SalarioVariable']
liquidaciones_garantizados_centro_costo['PagoReal'] = liquidaciones_garantizados_centro_costo['SalarioVariable']
liquidaciones_garantizados_centro_costo['PagoGarantizado'] = 'SI'

liquidaciones_garantizados_centro_costo.drop(columns=['DifMonths', 'FechaIngreso'], inplace=True)

liquidaciones_garantizados_centro_costo.head()

,Fecha,CodigoEmpleado,SalarioVariable,FactorMes,Liquidado,PagoReal,PagoGarantizado
4,2024-02-01,1015432392,500000.0,0,500000.0,500000.0,SI
5,2024-02-01,797582501,2000000.0,0,2000000.0,2000000.0,SI
8,2024-03-01,1015432392,500000.0,0,500000.0,500000.0,SI
9,2024-03-01,797582501,2000000.0,0,2000000.0,2000000.0,SI
10,2024-03-01,80037193,600000.0,0,600000.0,600000.0,SI


In [117]:
resultado_liquidaciones[(resultado_liquidaciones['CodigoEmpleado'] == '291169471') 
#| (resultado_liquidaciones['CodigoEmpleado'] == '1111135')
#| (resultado_liquidaciones['CodigoEmpleado'] == '16659046')
]

,CodigoEmpleado,SalarioVariable,Fecha,FactorMes,Liquidado,PagoReal,PagoGarantizado
261,291169471,6000000.0,2024-05-01,5.0,50850000.0,0.0,NO


## Se unen las liquidaciones de garantizados con las demás

### Sin centro de costo

In [118]:
resultado_liquidaciones = pd.concat([resultado_liquidaciones, liquidaciones_garantizados])
resultado_liquidaciones.reset_index(drop=True, inplace=True)
resultado_liquidaciones

,CodigoEmpleado,SalarioVariable,Fecha,FactorMes,Liquidado,PagoReal,PagoGarantizado
0,1000972927,NaN,2024-01-01,NaN,0.0,150000.0,NO
1,1000972927,NaN,2024-02-01,NaN,0.0,150000.0,NO
2,1000972927,NaN,2024-03-01,NaN,0.0,1218750.0,NO
3,1000972927,NaN,2024-04-01,NaN,0.0,1259860.0,NO
4,1000972927,388000.0,2024-05-01,5.0,3498790.0,0.0,NO
...,...,...,...,...,...,...,...
665,1019140853,1000000.0,2024-04-01,0.0,1000000.0,1000000.0,SI
666,1015432392,500000.0,2024-05-01,0.0,500000.0,500000.0,SI
667,797582501,2000000.0,2024-05-01,0.0,2000000.0,2000000.0,SI
668,80037193,600000.0,2024-05-01,0.0,600000.0,600000.0,SI


### Con centro de costo

In [119]:
resultado_liquidaciones_centro_costo = pd.concat([resultado_liquidaciones_centro_costo, liquidaciones_garantizados_centro_costo])
resultado_liquidaciones_centro_costo.reset_index(drop=True, inplace=True)
resultado_liquidaciones_centro_costo

,CodigoEmpleado,CodigoCentroCosto,SalarioVariable,Fecha,FactorMes,Liquidado,PagoReal,PagoGarantizado
0,1000972927,NaN,NaN,2024-01-01,NaN,0.0,150000.0,NO
1,1000972927,NaN,NaN,2024-02-01,NaN,0.0,150000.0,NO
2,1000972927,NaN,NaN,2024-03-01,NaN,0.0,1218750.0,NO
3,1000972927,NaN,NaN,2024-04-01,NaN,0.0,1259860.0,NO
4,1000972927,323007260,388000.0,2024-05-01,5.0,190120.0,0.0,NO
...,...,...,...,...,...,...,...,...
1063,1019140853,NaN,1000000.0,2024-04-01,0.0,1000000.0,1000000.0,SI
1064,1015432392,NaN,500000.0,2024-05-01,0.0,500000.0,500000.0,SI
1065,797582501,NaN,2000000.0,2024-05-01,0.0,2000000.0,2000000.0,SI
1066,80037193,NaN,600000.0,2024-05-01,0.0,600000.0,600000.0,SI


## Se conserva la información del histórico y se le añade la nueva información hasta la fecha de liquidación

### Sin centro de costo

In [120]:
resultado_liquidaciones_copy = resultado_liquidaciones.copy()
resultado_liquidaciones = resultado_liquidaciones_copy

In [121]:
def calcular_liquidacion_mes_corregido(g):

    suma_anticipos = 0
    meses_incentivos = [3,4,5,6,7,8,9,10,11,12]
    g = g.sort_values(by=['CodigoEmpleado', 'Fecha'])
    pago_acumulado = 0
    
    for i, row in g.iterrows():

        if row['Fecha'].month in meses_incentivos:
            if g.at[i, 'PagoReal'] == 0:
                if i == 0:
                    g.at[i, 'PagoReal'] = g.at[i, 'Liquidado']
                else:
                    g.at[i, 'PagoReal'] = g.at[i, 'Liquidado'] - pago_acumulado
            if g.at[i, 'PagoReal'] < 0:
                g.at[i, 'PagoReal'] = 0.0

        if row['Fecha'].month not in meses_incentivos:
            if g.at[i, 'PagoGarantizado'] == "NO":
                suma_anticipos += g.at[i, 'PagoReal']

        if row['Fecha'].month == 4:
            g.at[i, 'PagoReal'] = suma_anticipos + g.at[i, 'PagoReal']
            

        if g.at[i, 'PagoGarantizado'] == "NO":
            
            pago_acumulado += g.at[i, 'PagoReal']

        
        g.at[i, 'PagadoFecha'] = pago_acumulado
        
        excepcion = excepciones_pagos_acumulados.get(row['CodigoEmpleado'], None)
        if excepcion and row['Fecha'] == excepcion['Fecha']:
            pago_acumulado = excepcion['ValorPagoAcumulado']
        
        excepcion_diferencia = excepciones_diferencias.get(row['CodigoEmpleado'], None)
        if excepcion_diferencia and row['Fecha'] == excepcion_diferencia['Fecha']:
            pago_acumulado = pago_acumulado + excepcion_diferencia['ValorDiferencia']
        
    return g

In [122]:
resultado_liquidaciones_historico = df_resultado_liquidaciones_previo.copy()
resultado_liquidaciones_historico['PagoGarantizado'] = resultado_liquidaciones_historico['PagoGarantizado'].fillna('NO')
last_date_historico = resultado_liquidaciones_historico['Fecha'].max()
last_date_historico = None if pd.isnull(last_date_historico) else last_date_historico

try:
    resultado_liquidaciones = resultado_liquidaciones[
        resultado_liquidaciones['Fecha']<=fecha_liquidacion
    ]
    if last_date_historico:
        resultado_liquidaciones = resultado_liquidaciones[
            resultado_liquidaciones['Fecha']>last_date_historico
        ]
except:
    pass

resultado_liquidaciones = pd.concat([resultado_liquidaciones_historico, resultado_liquidaciones])
resultado_liquidaciones = resultado_liquidaciones.sort_values(by=['CodigoEmpleado', 'Fecha'])

for codigo_empleado_excepcion in excepciones_pagos_acumulados:
    excepcion = excepciones_pagos_acumulados[codigo_empleado_excepcion]
    resultado_liquidaciones['PagoReal'] = np.where(
        (
            (resultado_liquidaciones['CodigoEmpleado'] == codigo_empleado_excepcion) &
            (resultado_liquidaciones['Fecha'] <= excepcion['Fecha'])
        ),
        excepcion['ValorPagoAcumulado'],
        resultado_liquidaciones['PagoReal']
    )

for codigo_empleado_excepcion in excepciones_diferencias:
    excepcion = excepciones_diferencias[codigo_empleado_excepcion]
    resultado_liquidaciones['PagoReal'] = np.where(
        (
            (resultado_liquidaciones['CodigoEmpleado'] == codigo_empleado_excepcion) &
            (resultado_liquidaciones['Fecha'] == excepcion['Fecha'])
        ),
        resultado_liquidaciones['PagoReal'] + excepcion['ValorDiferencia'],
        resultado_liquidaciones['PagoReal']
    )
resultado_liquidaciones['PagadoFecha'] = 0
resultado_liquidaciones = resultado_liquidaciones.groupby(['CodigoEmpleado'], group_keys=False).apply(calcular_liquidacion_mes_corregido)
#resultado_liquidaciones['PagadoFecha'] = resultado_liquidaciones.groupby(['CodigoEmpleado'])['PagoReal'].transform(pd.Series.cumsum)
#resultado_liquidaciones['PagadoFecha'] = resultado_liquidaciones['PagadoFecha'] - resultado_liquidaciones['PagoReal']
resultado_liquidaciones['PagadoFecha'] = resultado_liquidaciones.apply(
    lambda row: row['PagadoFecha'] - row['PagoReal'] if row['PagoGarantizado'] == 'NO' else row['PagadoFecha'], axis=1
)
resultado_liquidaciones[(resultado_liquidaciones['CodigoEmpleado'] == '16659046') | (resultado_liquidaciones['CodigoEmpleado'] == '1111135') | (resultado_liquidaciones['CodigoEmpleado'] == '291169471')]
#resultado_liquidaciones

C:\Users\jjgg\AppData\Local\Temp\ipykernel_25464\974264508.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2778610.000000001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  g.at[i, 'PagadoFecha'] = pago_acumulado
C:\Users\jjgg\AppData\Local\Temp\ipykernel_25464\974264508.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '3008238.0000000005' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  g.at[i, 'PagadoFecha'] = pago_acumulado
C:\Users\jjgg\AppData\Local\Temp\ipykernel_25464\974264508.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '25812504.000000004' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  g.at[i, 'PagadoFecha'] = pago_acumulado
C:

,CodigoEmpleado,SalarioVariable,Fecha,FactorMes,PagoGarantizado,Liquidado,PagoReal,PagadoFecha
81,1111135,2500000.0,2024-01-01,0.0,SI,2500000.0,2500000.0,0.0
362,1111135,2500000.0,2024-03-01,2.0,NO,5611250.0,4361250.0,0.0
501,1111135,2500000.0,2024-04-01,3.0,NO,7653750.0,3292500.0,4361250.0
230,1111135,2500000.0,2024-05-01,5.0,NO,14312500.0,6658750.0,7653750.0
5,16659046,1712000.0,2024-01-01,1.0,NO,291040.0,856000.0,0.0
138,16659046,1712000.0,2024-02-01,2.0,NO,1712000.0,856000.0,856000.0
281,16659046,1712000.0,2024-03-01,3.0,NO,10272000.0,8560000.0,1712000.0
422,16659046,1712000.0,2024-04-01,4.0,NO,17398500.0,8838500.0,10272000.0
255,16659046,2286800.0,2024-05-01,5.0,NO,24011400.0,4900900.0,19110500.0
29,291169471,6000000.0,2024-01-01,0.0,SI,6000000.0,6000000.0,0.0


In [123]:
resultado_liquidaciones[(resultado_liquidaciones['CodigoEmpleado'] == '291169471') 
#| (resultado_liquidaciones['CodigoEmpleado'] == '1111135')
#| (resultado_liquidaciones['CodigoEmpleado'] == '16659046')
]

,CodigoEmpleado,SalarioVariable,Fecha,FactorMes,PagoGarantizado,Liquidado,PagoReal,PagadoFecha
29,291169471,6000000.0,2024-01-01,0.0,SI,6000000.0,6000000.0,0.0
308,291169471,6000000.0,2024-03-01,0.0,SI,6000000.0,6000000.0,0.0
449,291169471,6000000.0,2024-04-01,0.0,SI,6000000.0,6000000.0,0.0
261,291169471,6000000.0,2024-05-01,5.0,NO,50850000.0,50850000.0,0.0


In [124]:
resultado_liquidaciones.drop_duplicates(['CodigoEmpleado', 'Fecha'], inplace=True)

resultado_final = pd.merge(
    left=resultado_liquidaciones[['CodigoEmpleado', 'Fecha', 'PagoGarantizado', 'Liquidado', 'PagoReal', 'PagadoFecha', 'FactorMes']],
    right=df_empleados,
    on=['CodigoEmpleado'],
    how='left'
)
resultado_final = resultado_final[['Identificacion', 'Nombre', 'Apellidos', 'CodigoEmpleado', 'TipoEmpleado', 'AreaCalculo', 'Fecha', 'SalarioVariable', 'PagoGarantizado', 'FactorMes', 'Liquidado', 'PagoReal', 'PagadoFecha']]
resultado_final = resultado_final.sort_values(by=['TipoEmpleado', 'AreaCalculo', 'CodigoEmpleado', 'Fecha'])

resultado_final.rename(columns={
    'Identificacion': 'Cédula',
    'CodigoEmpleado': 'Código',
    'TipoEmpleado': 'Tipo Empleado',
    'AreaCalculo': 'Área de cálculo',
    'Liquidado': 'Valor liquidado',
    'PagoReal': 'Valor neto a pagar',
    'PagadoFecha': 'Pagado a la fecha',
    'SalarioVariable': 'Salario variable',
    'FactorMes': 'Factor mes',
    'PagoGarantizado': '¿Pago garantizado?'
}, inplace=True)
resultado_final.head()

,Cédula,Nombre,Apellidos,Código,Tipo Empleado,Área de cálculo,Fecha,Salario variable,¿Pago garantizado?,Factor mes,Valor liquidado,Valor neto a pagar,Pagado a la fecha
515,51790647,SONIA AHIDE,MARTIN AVILA,51790647,AD,1.0,2024-01-01,3392000.0,NO,1.0,576640.0,1696000.0,0.0
516,51790647,SONIA AHIDE,MARTIN AVILA,51790647,AD,1.0,2024-02-01,3392000.0,NO,2.0,3392000.0,1696000.0,1696000.0
517,51790647,SONIA AHIDE,MARTIN AVILA,51790647,AD,1.0,2024-03-01,3392000.0,NO,3.0,20352000.0,16960000.0,3392000.0
518,51790647,SONIA AHIDE,MARTIN AVILA,51790647,AD,1.0,2024-04-01,3392000.0,NO,4.0,34198500.0,17238500.0,20352000.0
519,51790647,SONIA AHIDE,MARTIN AVILA,51790647,AD,1.0,2024-05-01,3392000.0,NO,5.0,47027400.0,9436900.0,37590500.0


### Con centro de costo

In [125]:
resultado_liquidaciones_historico_centro_costo = df_resultado_liquidaciones_previo.copy()
resultado_liquidaciones_historico_centro_costo['PagoGarantizado'] = resultado_liquidaciones_historico_centro_costo['PagoGarantizado'].fillna('NO')
last_date_historico = resultado_liquidaciones_historico_centro_costo['Fecha'].max()
last_date_historico = None if pd.isnull(last_date_historico) else last_date_historico

try:
    resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo[
        resultado_liquidaciones['Fecha']<=fecha_liquidacion
    ]
    if last_date_historico:
        resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo[
            resultado_liquidaciones_centro_costo['Fecha']>last_date_historico
        ]
except:
    pass

resultado_liquidaciones_centro_costo = pd.concat([resultado_liquidaciones_historico_centro_costo, resultado_liquidaciones_centro_costo])
resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo.sort_values(by=['CodigoEmpleado', 'Fecha'])

for codigo_empleado_excepcion in excepciones_pagos_acumulados:
    excepcion = excepciones_pagos_acumulados[codigo_empleado_excepcion]
    resultado_liquidaciones_centro_costo['PagoReal'] = np.where(
        (
            (resultado_liquidaciones_centro_costo['CodigoEmpleado'] == codigo_empleado_excepcion) &
            (resultado_liquidaciones_centro_costo['Fecha'] <= excepcion['Fecha'])
        ),
        excepcion['ValorPagoAcumulado'],
        resultado_liquidaciones_centro_costo['PagoReal']
    )

for codigo_empleado_excepcion in excepciones_diferencias:
    excepcion = excepciones_diferencias[codigo_empleado_excepcion]
    resultado_liquidaciones_centro_costo['PagoReal'] = np.where(
        (
            (resultado_liquidaciones_centro_costo['CodigoEmpleado'] == codigo_empleado_excepcion) &
            (resultado_liquidaciones_centro_costo['Fecha'] == excepcion['Fecha'])
        ),
        resultado_liquidaciones_centro_costo['PagoReal'] + excepcion['ValorDiferencia'],
        resultado_liquidaciones_centro_costo['PagoReal']
    )

resultado_liquidaciones_centro_costo['PagadoFecha'] = resultado_liquidaciones_centro_costo.groupby(['CodigoEmpleado'])['PagoReal'].transform(pd.Series.cumsum)
resultado_liquidaciones_centro_costo['PagadoFecha'] = resultado_liquidaciones_centro_costo['PagadoFecha'] - resultado_liquidaciones_centro_costo['PagoReal']
resultado_liquidaciones_centro_costo

C:\Users\jjgg\AppData\Local\Temp\ipykernel_25464\2517264680.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  resultado_liquidaciones_centro_costo = resultado_liquidaciones_centro_costo[


,CodigoEmpleado,SalarioVariable,Fecha,FactorMes,PagoGarantizado,Liquidado,PagoReal,CodigoCentroCosto,PagadoFecha
51,1000972927,300000.0,2024-01-01,1.0,NO,200700.000,150000.000,NaN,0.000000e+00
0,1000972927,NaN,2024-01-01,NaN,NO,0.000,150000.000,NaN,1.500000e+05
186,1000972927,300000.0,2024-02-01,2.0,NO,300000.000,150000.000,NaN,3.000000e+05
1,1000972927,NaN,2024-02-01,NaN,NO,0.000,150000.000,NaN,4.500000e+05
330,1000972927,300000.0,2024-03-01,3.0,NO,1518750.000,1218750.000,NaN,6.000000e+05
...,...,...,...,...,...,...,...,...,...
545,94487610,2000000.0,2024-04-01,4.0,NO,8247695.625,3369695.625,NaN,9.756000e+06
1052,94487610,NaN,2024-04-01,NaN,NO,0.000,5369695.625,NaN,1.312570e+07
1053,94487610,2154000.0,2024-05-01,5.0,NO,4462818.750,0.000,323015045,1.849539e+07
244,9990010,0.0,2024-02-01,2.0,NO,0.000,0.000,NaN,0.000000e+00


In [126]:
resultado_liquidaciones_centro_costo.drop_duplicates(['CodigoEmpleado', 'Fecha'], inplace=True)

resultado_final_centro_costo = pd.merge(
    left=resultado_liquidaciones_centro_costo[['CodigoEmpleado', 'Fecha', 'PagoGarantizado', 'Liquidado', 'PagoReal','PagadoFecha', 'FactorMes', 'CodigoCentroCosto']],
    right=df_empleados,
    on=['CodigoEmpleado'],
    how='left'
)
resultado_final_centro_costo = resultado_final_centro_costo[['Identificacion', 'Nombre', 'Apellidos', 'CodigoEmpleado', 'TipoEmpleado', 'AreaCalculo', 'Fecha', 'SalarioVariable', 'PagoGarantizado', 'FactorMes', 'Liquidado', 'PagoReal', 'PagadoFecha', 'CodigoCentroCosto']]
resultado_final_centro_costo = resultado_final_centro_costo.sort_values(by=['TipoEmpleado', 'AreaCalculo', 'CodigoEmpleado', 'Fecha', 'CodigoCentroCosto'])

resultado_final_centro_costo.rename(columns={
    'Identificacion': 'Cédula',
    'CodigoEmpleado': 'Código',
    'TipoEmpleado': 'Tipo Empleado',
    'AreaCalculo': 'Área de cálculo',
    'Liquidado': 'Valor liquidado',
    'PagoReal': 'Valor neto a pagar',
    'PagadoFecha': 'Pagado a la fecha',
    'SalarioVariable': 'Salario variable',
    'FactorMes': 'Factor mes',
    'PagoGarantizado': '¿Pago garantizado?'
}, inplace=True)
resultado_final_centro_costo.head()

,Cédula,Nombre,Apellidos,Código,Tipo Empleado,Área de cálculo,Fecha,Salario variable,¿Pago garantizado?,Factor mes,Valor liquidado,Valor neto a pagar,Pagado a la fecha,CodigoCentroCosto
519,51790647,SONIA AHIDE,MARTIN AVILA,51790647,AD,1.0,2024-01-01,3392000.0,NO,1.0,576640.0,1696000.0,0.0,NaN
520,51790647,SONIA AHIDE,MARTIN AVILA,51790647,AD,1.0,2024-02-01,3392000.0,NO,2.0,3392000.0,1696000.0,3392000.0,NaN
521,51790647,SONIA AHIDE,MARTIN AVILA,51790647,AD,1.0,2024-03-01,3392000.0,NO,3.0,20352000.0,16960000.0,6784000.0,NaN
522,51790647,SONIA AHIDE,MARTIN AVILA,51790647,AD,1.0,2024-04-01,3392000.0,NO,4.0,34198500.0,13846500.0,40704000.0,NaN
523,51790647,SONIA AHIDE,MARTIN AVILA,51790647,AD,1.0,2024-05-01,3392000.0,NO,5.0,22394000.0,0.0,71789000.0,323004010


In [127]:
resultado_liquidaciones[resultado_liquidaciones['CodigoEmpleado'] == '16659046']

,CodigoEmpleado,SalarioVariable,Fecha,FactorMes,PagoGarantizado,Liquidado,PagoReal,PagadoFecha
5,16659046,1712000.0,2024-01-01,1.0,NO,291040.0,856000.0,0.0
138,16659046,1712000.0,2024-02-01,2.0,NO,1712000.0,856000.0,856000.0
281,16659046,1712000.0,2024-03-01,3.0,NO,10272000.0,8560000.0,1712000.0
422,16659046,1712000.0,2024-04-01,4.0,NO,17398500.0,8838500.0,10272000.0
255,16659046,2286800.0,2024-05-01,5.0,NO,24011400.0,4900900.0,19110500.0


In [128]:
resultado_final_centro_costo.to_excel(os.path.join(OUTPUT_FOLDER, 'resultado_final_centro_costo.xlsx'))
maestra_resultados_por_empleado.to_excel(os.path.join(OUTPUT_FOLDER, 'maestra_resultados_por_empleado.xlsx'))
maestra_resultados_por_empleado_centro_costo.to_excel(os.path.join(OUTPUT_FOLDER, 'maestra_resultados_por_empleado_centro_costo.xlsx'))
resultado_liquidaciones.to_excel(os.path.join(OUTPUT_FOLDER, 'resultado_liquidaciones.xlsx'))
resultado_final.to_excel(os.path.join(OUTPUT_FOLDER, 'resultado_final.xlsx'))
maestra_resultados.to_excel(os.path.join(OUTPUT_FOLDER, 'maestra_resultados.xlsx'))

In [129]:
maestra_resultados_por_empleado_centro_costo[
    (maestra_resultados_por_empleado_centro_costo['CodigoEmpleado'] == '5002899')
]

,Contexto,Fecha,GrupoProducto,Division,CodigoCentroCosto,Real,Presupuesto,Variable,PorcentajeCumplimiento,RealCOP,...,Porcentaje,Consecutivo,Nombre variable,Nombre consecutivo,TipoEmpleado,AreaCalculo,Apellidos,Nombre,FechaIngreso,SalarioVariable


## Generación de PDFs

In [130]:
def get_dataframes(
        df_resultado_detallado,
        df_resultado_liquidaciones
):
    resultado_liquidaciones = df_resultado_liquidaciones.set_index(['CodigoEmpleado', 'Fecha'])
    resultado_liquidaciones.fillna(value={
        'PagoReal': 0,
        'Liquidacion': 0
    }, inplace=True)

    resultado_detallado = df_resultado_detallado.fillna(value={
        'FactorIncentivo': 0,
        'PorcentajeCumplimiento': 0,
        'Presupuesto': 0,
        'Real': 0,
        'Liquidado': 0,
        'ResultadoPresupuesto': 0,
        'ResultadoRealMes': 0,
        'PorcentajeCumplimientoMes': 0
    })

    return resultado_detallado, resultado_liquidaciones

In [131]:
df_detalles_para_pdf, df_liquidaciones_para_pdf = get_dataframes(
    df_resultado_detallado=maestra_resultados_por_empleado,
    df_resultado_liquidaciones=resultado_liquidaciones
)